# Import bibliotek

In [2]:
# --- Przetwarzanie danych ---
import numpy as np
import pandas as pd

# --- Wizualizacja ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Preprocessing ---
from scipy.signal import correlate2d
from scipy.stats import bartlett, levene, fligner
from skimage.metrics import structural_similarity, variation_of_information, normalized_root_mse
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.metrics import (
    silhouette_score,
    silhouette_samples,
    homogeneity_completeness_v_measure,
    adjusted_rand_score,
    adjusted_mutual_info_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    fowlkes_mallows_score,
    mean_squared_error,
)
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
import umap

# --- Modelowanie ---
import optuna
import optunahub
from sklearn.cluster import FeatureAgglomeration, KMeans
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import (
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    VotingClassifier,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# --- Ewaluacja ---
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    make_scorer,
)
from sklearn.model_selection import cross_val_score

# --- Przetwarzanie obrazów ---
import cv2 as cv2
from skimage.color import rgb2gray
from skimage.feature import hog, graycomatrix, graycoprops
from skimage.filters import gabor_kernel, gabor
from skimage.measure import label, regionprops
from skimage import img_as_float32
from skimage.transform import resize

# --- Zrównoleglanie ---
from joblib import Parallel, delayed
from multiprocessing import Pool

# --- Inne ---
import time
import gc
import warnings
import sqlite3

# --- Sklearn ---
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

# --- Skimage ---
from skimage.transform import AffineTransform, warp
from skimage.filters import gaussian, unsharp_mask
from skimage.measure import find_contours

# --- Dodatkowe importy ---
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import FunctionTransformer



train = pd.read_csv('mnist_train.csv')
test  = pd.read_csv('mnist_test.csv')
train.info()
test.info()

# Podział na cechy i etykiety
X_train = train.drop('label', axis=1).values.reshape(-1, 28, 28)
y_train = train['label'].values

X_test = test.drop('label', axis=1).values.reshape(-1, 28, 28)
y_test = test['label'].values

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 59.9 MB
Shape of X_train: (60000, 28, 28)
Shape of y_train: (60000,)
Shape of X_test: (10000, 28, 28)
Shape of y_test: (10000,)


# EDA

In [ ]:
train = pd.read_csv('mnist_train.csv')
test  = pd.read_csv('mnist_test.csv')
train.info()
test.info()

# Podział na cechy i etykiety
X_train = train.drop('label', axis=1).values.reshape(-1, 28, 28)
y_train = train['label'].values

X_test = test.drop('label', axis=1).values.reshape(-1, 28, 28)
y_test = test['label'].values

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

 ### Wizualizacja podstawowych cech

In [ ]:
# Funkcja do wizualizacji statystyk
def visualize_statistics(X, label):
    """Oblicza i wizualizuje statystyki dla danej klasy."""

    # Obliczenie statystyk
    mean_pixels = np.mean(X, axis=0)
    std_pixels = np.std(X, axis=0)
    median_pixels = np.median(X, axis=0)
    q25_pixels = np.quantile(X, 0.25, axis=0)
    q75_pixels = np.quantile(X, 0.75, axis=0)
    min_pixels = np.mean(np.percentile(X, [0, 1.5], axis=0), axis=0)
    max_pixels = np.mean(np.percentile(X, [98.5, 100], axis=0), axis=0)

    # Wizualizacja wszystkich statystyk na jednym wykresie
    _, axes = plt.subplots(2, 4, figsize=(15, 8))

    sns.heatmap(mean_pixels, cmap='gray', ax=axes[0, 0])
    axes[0, 0].set_title(f"Średnia")

    sns.heatmap(std_pixels, cmap='viridis', ax=axes[0, 1])
    axes[0, 1].set_title(f"Odchylenie std.")

    axes[0, 2].imshow(median_pixels, cmap='gray')
    axes[0, 2].set_title("Mediana")
    axes[0, 2].axis('off')

    axes[0, 3].imshow(q25_pixels, cmap='gray')
    axes[0, 3].set_title("Kwartyl 25%")
    axes[0, 3].axis('off')

    axes[1, 0].imshow(q75_pixels, cmap='gray')
    axes[1, 0].set_title("Kwartyl 75%")
    axes[1, 0].axis('off')

    axes[1, 1].imshow(min_pixels, cmap='gray')
    axes[1, 1].set_title("1.5% minimum")
    axes[1, 1].axis('off')

    axes[1, 2].imshow(max_pixels, cmap='gray')
    axes[1, 2].set_title("1.5% maksimum")
    axes[1, 2].axis('off')

    # Histogram wartości pikseli
    axes[1, 3].hist(X.flatten(), bins=50)
    axes[1, 3].set_title(f"Histogram\nwartości pikseli")
    axes[1, 3].set_xlabel('Wartość piksela')
    axes[1, 3].set_ylabel('Liczba wystąpień')

    plt.suptitle(f"Etykieta: {label}", fontsize=14)
    plt.tight_layout()
    plt.show()

# Grupowanie po etykietach w zbiorze treningowym
for label in range(10):
    X_train_group = X_train[y_train == label]
    X_test_group = X_test[y_test == label]

    # Wizualizacja statystyk dla zbioru treningowego
    print("Zbiór treningowy:")
    visualize_statistics(X_train_group, label)

    # Wizualizacja statystyk dla zbioru testowego
    print("\nZbiór testowy:")
    visualize_statistics(X_test_group, label)

In [ ]:
def porownaj_srednie_obrazow(X_train, y_train, X_test, y_test):
    """
    Porównuje średnie obrazy dla każdej klasy w zbiorze treningowym i testowym 
    za pomocą trzech metryk: SSIM, NRMSE i Variation of Information.

    Args:
        X_train: Zbiór treningowy obrazów (numpy array).
        y_train: Etykiety dla zbioru treningowego (numpy array).
        X_test: Zbiór testowy obrazów (numpy array).
        y_test: Etykiety dla zbioru testowego (numpy array).

    Returns:
        Słownik zawierający wyniki dla każdej metryki i klasy.
    """

    wyniki = {}
    for label in range(10):
        X_train_group = X_train[y_train == label]
        X_test_group = X_test[y_test == label]

        # Obliczenie średnich obrazów
        sredni_obraz_train = np.mean(X_train_group, axis=0)
        sredni_obraz_test = np.mean(X_test_group, axis=0)

        # Obliczenie metryk
        ssim = structural_similarity(sredni_obraz_train, sredni_obraz_test, data_range=sredni_obraz_train.max() - sredni_obraz_train.min())
        nrmse = normalized_root_mse(sredni_obraz_train, sredni_obraz_test)

        # Variation of Information wymaga obrazów typu int
        vi = variation_of_information(sredni_obraz_train.astype(int), sredni_obraz_test.astype(int)) 

        wyniki[label] = {
            'ssim': ssim,
            'nrmse': nrmse,
            'vi': vi,
        }

    return wyniki


wyniki = porownaj_srednie_obrazow(X_train, y_train, X_test, y_test)


plt.figure(figsize=(12, 6))

# SSIM
plt.subplot(1, 3, 1)
sns.barplot(x=list(wyniki.keys()), y=[wyniki[label]['ssim'] for label in wyniki])
plt.title('SSIM')
plt.xlabel('Klasa')
plt.ylabel('Wartość SSIM')

# NRMSE
plt.subplot(1, 3, 2)
sns.barplot(x=list(wyniki.keys()), y=[wyniki[label]['nrmse'] for label in wyniki])
plt.title('NRMSE')
plt.xlabel('Klasa')
plt.ylabel('Wartość NRMSE')

# VI
plt.subplot(1, 3, 3)
sns.barplot(x=list(wyniki.keys()), y=[wyniki[label]['vi'][0] for label in wyniki])
plt.title('Variation of Information')
plt.xlabel('Klasa')
plt.ylabel('Wartość VI')

plt.tight_layout()
plt.show()

### Wizualizacja wartości i wektrów własnych

In [ ]:
def analyze_eigen(X, label):
    """Analizuje i wizualizuje wartości i wektory własne dla danej klasy."""

    # Spłaszczenie danych
    X_flat = X.reshape(X.shape[0], -1)

    # Obliczenie macierzy kowariancji
    cov_matrix = np.cov(X_flat, rowvar=False)

    # Obliczenie wartości i wektorów własnych
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sortowanie wartości własnych
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Utworzenie siatki dla wykresów
    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(2, 5, height_ratios=[3, 1])  

    # Wykres wartości własnych z podwójnymi osiami Y
    ax1 = fig.add_subplot(gs[0, :])  
    color = 'tab:blue'
    ax1.set_xlabel('Numer wartości własnej')
    ax1.set_ylabel('Wartość własna', color=color)
    ax1.plot(eigenvalues, marker='o', color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True)
    ax1.set_title(f"Wektory własne i wartości własne dla etykiety {label}")

    # Utworzenie drugiej osi Y (prawa oś Y)
    ax2 = ax1.twinx()

    # Wykres wartości własnych od 200
    color = 'tab:red'
    ax2.set_ylabel('Wartość własna (powiększenie)', color=color)
    ax2.plot(range(200, len(eigenvalues)), eigenvalues[200:], marker='o', color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    # Wizualizacja wektorów własnych w drugim wierszu
    num_eigenvectors_to_display = 5
    for i in range(num_eigenvectors_to_display):
        ax = fig.add_subplot(gs[1, i])
        ax.imshow(eigenvectors[:, i].reshape(28, 28), cmap='gray')
        ax.set_title(f"Wektor własny {i+1}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Iteracja po etykietach (zakładając, że masz X_train i y_train)
for label in range(10):
    X_train_group = X_train[y_train == label]
    analyze_eigen(X_train_group, label)


### Metody składowych głównych

In [ ]:
def reduce_dimensionality(X_train, y_train, method_name):
    """
    Redukuje wymiarowość danych MNIST za pomocą określonej metody i ocenia jej jakość.

    Args:
      X_train: Dane treningowe MNIST.
      y_train: Etykiety danych treningowych.
      method_name: Nazwa metody redukcji wymiarów.

    Returns:
      Krotka (nazwa metody, dane po redukcji wymiarów, czas obliczeń).
    """

    X_train_flat = X_train.reshape(X_train.shape[0], -1)

    start_time = time.time()

    if method_name == "PCA":
        method = PCA(n_components=2)
        X_transformed = method.fit_transform(X_train_flat)
        
    elif method_name == "KernelPCA":
        method = KernelPCA(n_components=2, kernel="cosine")
        X_transformed = method.fit_transform(X_train_flat)
        
    elif method_name == "t-SNE":
        method = TSNE(
            n_components=2, random_state=42, perplexity=30)   # 3>method='exact',
        X_transformed = method.fit_transform(X_train_flat)

    elif method_name == "UMAP": 
        method = umap.UMAP(n_components=2, random_state=42)
        X_transformed = method.fit_transform(X_train_flat)
        
    elif method_name == "LLE":
        method = LocallyLinearEmbedding(n_components=2)
        X_transformed = method.fit_transform(X_train_flat)
        
    elif method_name == "LDA":
        method = LinearDiscriminantAnalysis(n_components=2)
        X_transformed = method.fit_transform(X_train_flat, y_train)
        
    else:
        raise ValueError(f"Nieznana metoda: {method_name}")

    end_time = time.time()
    time_taken = end_time - start_time

    return method_name, X_transformed, time_taken


def evaluate_and_visualize(X_train, y_train, method_name, X_transformed, time_taken):
    """
    Ocenia jakość redukcji wymiarów i wizualizuje wyniki dla pojedynczej metody.

    Args:
      X_train: Dane treningowe MNIST.
      y_train: Etykiety danych treningowych.
      method_name: Nazwa metody redukcji wymiarów.
      X_transformed: Dane po redukcji wymiarów.
      time_taken: Czas obliczeń.
    """
    
    # --- Wizualizacja ---
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(
        X_transformed[:, 0],
        X_transformed[:, 1],
        c=y_train,
        cmap="jet",
        alpha=0.5,
    )
    plt.title(f"{method_name} MNIST")
    plt.xlabel(f"{method_name} wymiar 1")
    plt.ylabel(f"{method_name} wymiar 2")
    legend = plt.legend(
        *scatter.legend_elements(), loc="lower left", title="Cyfry"
    )
    plt.gca().add_artist(legend)
    plt.tight_layout()
    plt.show()

    # --- Ocena jakości redukcji wymiarów ---
    if method_name != "LDA":
        silhouette_score_value = silhouette_score(X_transformed, y_train ,sample_size=10000)
        print(f"{method_name}: silhouette score = {silhouette_score_value:.3f}")
        gc.collect()
    print(f"{method_name}: czas obliczeń = {time_taken:.2f} s")
    
    # homogeneity and completeness and V-Measure
    # Klasteryzacja 
    kmeans = KMeans(n_clusters=10, random_state=42) 
    y_pred = kmeans.fit_predict(X_transformed)

    # homogeneity_completeness_v_measure
    homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(
        y_train, y_pred
    )

    print(f"{method_name}: homogeneity = {homogeneity:.3f}")
    print(f"{method_name}: completeness = {completeness:.3f}")
    print(f"{method_name}: v_measure = {v_measure:.3f}")

    print(f"{method_name}: czas obliczeń = {time_taken:.2f} s")

# Uruchomienie metod redukcji wymiarów sekwencyjnie
method_names = ["PCA", "KernelPCA", "LLE", "LDA", "t-SNE", "UMAP",]

for method_name in method_names:
    method_name, X_transformed, time_taken = reduce_dimensionality(
        X_train, y_train, method_name
    )
    evaluate_and_visualize(
        X_train, y_train, method_name, X_transformed, time_taken
    )

### Wyznacznik macierzy kowariancji, oraz korelacji przestrzennej dla średnich obrazów.


In [ ]:
def analyze_covariance(X):
    """
    Analizuje macierz kowariancji danych, obliczając wariancję, wyznacznik i liczbę warunkową.
    """

    variance = np.var(X, axis=0)  # Wariancja dla każdej cechy
    total_variance = np.sum(variance)  # Całkowita wariancja

    cov_matrix = np.cov(X, rowvar=False)
    determinant = np.linalg.det(cov_matrix)
    eigenvalues = np.linalg.eigvalsh(cov_matrix)

    # Usunięcie wartości własnych mniejszych niż 1e-8
    eigenvalues = eigenvalues[eigenvalues > 1e-8]

    condition_number = np.max(eigenvalues) / np.min(eigenvalues)

    return total_variance, determinant, condition_number

# Obliczenie średniego obrazu dla każdej klasy
mean_images = []
for label in range(10):
    mean_images.append(np.mean(X_train[y_train == label], axis=0))

# Obliczenie macierzy korelacji między średnimi obrazami
corr_matrix = np.corrcoef(np.array(mean_images).reshape(10, -1))

# Zastosowanie UMAP do całego zbioru danych
X_train_flat = X_train.reshape(X_train.shape[0], -1)
reducer = umap.UMAP(n_components=2, random_state=42)
X_train_reduced = reducer.fit_transform(X_train_flat)

# Wizualizacja macierzy korelacji i wyników analizy kowariancji
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Macierz korelacji
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", ax=axes[0])
axes[0].set_title('Macierz korelacji między średnimi obrazami dla danych nie przerobionych')
axes[0].set_xticks(np.arange(10) + 0.5)
axes[0].set_xticklabels(range(10))
axes[0].set_yticks(np.arange(10) + 0.5)
axes[0].set_yticklabels(range(10))

# Wyniki analizy kowariancji
determinants = []
condition_numbers = []
variances = []
silhouette_scores = []

for label in range(10):
    X_train_group_reduced = X_train_reduced[y_train == label]
    variance, determinant, condition_number = analyze_covariance(X_train_group_reduced)
    determinants.append(determinant)
    condition_numbers.append(condition_number)
    variances.append(variance)

    # Obliczenie silhouette score dla każdej klasy
    silhouette_avg = np.mean(silhouette_samples(X_train_reduced, y_train))
    silhouette_scores.append(silhouette_avg)

# --- KMeans clustering ---
kmeans = KMeans(n_clusters=10, random_state=42)
y_pred = kmeans.fit_predict(X_train_reduced)

# --- Metrics ---
y_true = y_train 

# Adjusted Rand Score
ars = adjusted_rand_score(y_true, y_pred)

# Adjusted Mutual Information Score
amis = adjusted_mutual_info_score(y_true, y_pred)

# Fowlkes-Mallows Score
fms = fowlkes_mallows_score(y_true, y_pred)

# Davies-Bouldin Index
dbi = davies_bouldin_score(X_train_reduced, y_pred)

# Calinski-Harabasz Index
chi = calinski_harabasz_score(X_train_reduced, y_pred)

print("UMAP: silhouette score = {:.3f}".format(silhouette_score(X_train_reduced, y_train)))
print("UMAP: adjusted_rand_score = {:.3f}".format(ars))
print("UMAP: adjusted_mutual_info_score = {:.3f}".format(amis))
print("UMAP: fowlkes_mallows_score = {:.3f}".format(fms))
print("UMAP: Davies-Bouldin index = {:.3f}".format(dbi))
print("UMAP: Calinski-Harabasz index = {:.3f}".format(chi))
print("UMAP: czas obliczeń = {:.2f} s".format(36.71))

# Wykresy
axes[1].bar(range(10), determinants, label='Wyznacznik')
axes[1].set_xticks(np.arange(10))
axes[1].set_ylabel('Wyznacznik')
axes[1].legend(loc='upper left')

ax2 = axes[1].twinx()
ax2.plot(range(10), condition_numbers, color='red', marker='o', label='Liczba warunkowa')
ax2.set_ylabel('Liczba warunkowa')
ax2.legend(loc='upper right')

axes[1].set_title('Wyznacznik i liczba warunkowa macierzy kowariancji po UMAP')

plt.tight_layout()
plt.show()

### Testy statystyczne

In [ ]:
def analyze_covariance(X, method='bartlett'):
    """
    Analizuje kowariancję w zbiorze danych X za pomocą wybranego testu.

    Dostępne testy:
    - 'bartlett': Test Bartletta 
    - 'levene': Test Levene'a
    - 'fligner': Test Flignera-Killeena

    Args:
        X: Zbiór danych (numpy array).
        method: Nazwa testu do wykonania.

    Returns:
        tuple: Statystyka testu i p-wartość.
    """

    if method == 'bartlett':
        statistic, p_value = bartlett(*X.T)
    elif method == 'levene':
        statistic, p_value = levene(*X.T)
    elif method == 'fligner':
        statistic, p_value = fligner(*X.T)
    else:
        raise ValueError("Nieznana metoda. Dostępne metody: 'bartlett', 'levene', 'fligner'")

    return statistic, p_value

# Zastosowanie UMAP do całego zbioru danych
X_train_flat = X_train.reshape(X_train.shape[0], -1)
reducer = umap.UMAP(n_components=2, random_state=42)
X_train_reduced = reducer.fit_transform(X_train_flat)  

grouped = pd.DataFrame(X_train_reduced).groupby(y_train)

results = [] 

# Iteracja po grupach i wykonanie testów
for label, group in grouped:
    for method in ['bartlett', 'levene', 'fligner']:
        statistic, p_value = analyze_covariance(group.values, method=method)

        # Dodanie wyników do listy jako słownik
        results.append({
            'label': label,
            'method': method,
            'statistic': statistic,
            'p_value': p_value
        })

results_df = pd.DataFrame(results)
pivot_table = results_df.pivot(index='label', columns='method', values=['statistic', 'p_value'])

display(pivot_table)

# Preprocessing

### One hot encoding

In [3]:
# Reshape 
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

# One-hot encoding
encoder = OneHotEncoder()
y_train_ENC = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_ENC = encoder.transform(y_test.reshape(-1, 1)).toarray()

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_train_ENC:", y_train_ENC.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of y_test_ENC:", y_test_ENC.shape)

Shape of X_train: (60000, 28, 28)
Shape of y_train: (60000,)
Shape of y_train_ENC: (60000, 10)
Shape of X_test: (10000, 28, 28)
Shape of y_test: (10000,)
Shape of y_test_ENC: (10000, 10)


### Normalizacja histogramem

In [4]:
X_train = cv2.normalize(X_train, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_64F)
X_train = X_train.astype(np.uint8)
# Wyrównywanie histogramu
for i in range(X_train.shape[0]):
    X_train[i] = cv2.equalizeHist(X_train[i])

X_test = cv2.normalize(X_test, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_64F)
X_test = X_test.astype(np.uint8)
for i in range(X_test.shape[0]):
    X_test[i] = cv2.equalizeHist(X_test[i])

### Powiększanie i wygładzenie obrazu

In [5]:
def enlarge_and_smooth(image, new_size):
    """Powiększa obraz za pomocą interpolacji bicubic."""
    enlarged_image = cv2.resize(image, new_size, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

# # Sztuczne zwiększenie rozmiaru obrazu 
# sample_indices = np.random.choice(X_train.shape[0], 5, replace=False)

# # Dla każdego wybranego obrazu
# for idx in sample_indices:
#     # Oryginalny obraz
#     original_image = X_train[idx]

#     # Powiększony i wygładzony obraz
#     enlarged_smoothed_image = enlarge_and_smooth(original_image, new_size=(42, 42))

#     # Wyświetl oryginalny i przekształcony obraz obok siebie
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
#     print("Shape of original_image:", original_image.shape)
#     print("Shape of enlarged_smoothed_image:", enlarged_smoothed_image.shape)

#     sns.heatmap(original_image, cmap="viridis", ax=axes[0]) 
#     axes[0].set_title(f'Oryginalny obraz ({idx})')

#     sns.heatmap(enlarged_smoothed_image, cmap="viridis", ax=axes[1]) 
#     axes[1].set_title(f'Powiększony i wygładzony obraz (Lanczos4) ({idx})')
#     plt.show()

### Train --- Test split na danych treningowych

In [6]:
# Baseline accuracy without transformations
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(
    X_train, y_train, test_size=0.2, random_state=69)
print("Rozmiar X_train_base:", X_train_base.shape)
print("Rozmiar X_val_base:", X_val_base.shape)
print("Rozmiar y_train_base:", y_train_base.shape)
print("Rozmiar y_val_base:", y_val_base.shape)


smoothed_X_train = np.array([enlarge_and_smooth(image, new_size=(42, 42)) for image in X_train])
# Baseline accuracy upscaling
X_train_Upsc, X_val_Upsc, y_train_Upsc, y_val_Upsc = train_test_split(
    smoothed_X_train, y_train, test_size=0.2, random_state=69)
print("Rozmiar X_train_Upsc:", X_train_Upsc.shape)
print("Rozmiar X_val_Upsc:", X_val_Upsc.shape)
print("Rozmiar y_train_Upsc:", y_train_Upsc.shape)
print("Rozmiar y_val_Upsc:", y_val_Upsc.shape)


Rozmiar X_train_base: (48000, 28, 28)
Rozmiar X_val_base: (12000, 28, 28)
Rozmiar y_train_base: (48000,)
Rozmiar y_val_base: (12000,)
Rozmiar X_train_Upsc: (48000, 42, 42)
Rozmiar X_val_Upsc: (12000, 42, 42)
Rozmiar y_train_Upsc: (48000,)
Rozmiar y_val_Upsc: (12000,)


## Klasa Ewaulacji modelu

In [7]:
def flatten(X):
    return X.reshape(X.shape[0], -1)

flatten_transformer = FunctionTransformer(flatten)

class ModelEvaluator:
    """
    Klasa do ewaluacji modeli klasyfikacji. Umożliwia
    przekazanie pipeline'u z dowolnymi operacjami
    przetwarzania danych. Oblicza i wyświetla wybrane
    metryki, macierz pomyłek oraz raport klasyfikacji.
    """

    def __init__(self, metrics=None, display_labels=None):
        """
        Inicjalizuje ModelEvaluator z domyślnymi lub
        zdefiniowanymi metrykami i etykietami.

        Args:
            metrics: Lista metryk do obliczenia.
                    Domyślnie: ["accuracy", "precision", "recall", "f1"].
                    Dostępne metryki: "accuracy", "precision", "recall", "f1", "roc_auc".
            display_labels: Etykiety klas do wyświetlenia w macierzy pomyłek.
                            Domyślnie None (używane unikalne etykiety z y_true).
        """
        self.metrics = metrics if metrics is not None else [
            "accuracy", "precision", "recall", "f1"
        ]
        self.display_labels = display_labels

    def evaluate(
        self,
        pipeline,
        X_train,
        y_train,
        X_val,
        y_val,
        calculate_proba=False,
        verbose=True,
    ):
        """
        Ewaluuje pipeline na podanych danych. Pipeline
        może zawierać dowolne operacje przetwarzania
        danych.

        Args:
            pipeline: Pipeline do ewaluacji.
            X_train: Dane treningowe.
            y_train: Etykiety treningowe.
            X_val: Dane walidacyjne.
            y_val: Etykiety walidacyjne.
            calculate_proba: Flaga wskazująca, czy obliczyć
                            prawdopodobieństwa (wymagane dla
                            roc_auc, domyślnie False).
            verbose: Flaga wskazująca, czy wyświetlać
                     szczegółowe wyniki (macierz pomyłek,
                     raport klasyfikacji, krzywa ROC,
                     domyślnie True).

        Returns:
            dict: Słownik z wynikami ewaluacji.
        """
        # Sprawdzenie, czy w pipeline jest redukcja wymiarów
        for i, step in enumerate(pipeline.steps):
            try:
                X_temp = step[1].fit_transform(X_train)  
                if X_temp.shape[1] < X_train.shape[1]:  
                    pipeline.steps.insert(i, ("flatten", flatten_transformer)) 
                    break
            except:
                pass  

        # Jeśli nie znaleziono redukcji wymiarów,  spłaszczenie na początku
        else:
            pipeline = Pipeline([
                ("flatten", flatten_transformer),
                *pipeline.steps
            ])

        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_val)

        y_prob = None
        if calculate_proba:
            try:
                y_prob = pipeline.predict_proba(X_val)[:, 1]
            except AttributeError:
                print(
                    "Model nie posiada metody predict_proba. "
                    "Metryka AUC nie zostanie obliczona."
                )

        eval_results = self._calculate_metrics(y_val, y_pred, y_prob, verbose, calculate_proba)
        self._display_results(pipeline, y_val, y_pred, eval_results, verbose, calculate_proba)  
        return eval_results

    def _calculate_metrics(self, y_true, y_pred, y_prob=None, verbose=True, calculate_proba=False):  
        """Oblicza wybrane metryki."""

        results = {}

        if not calculate_proba:  # Obliczanie tylko podstawowych metryk, gdy calculate_proba=False
            results["accuracy"] = accuracy_score(y_true, y_pred)
            results["precision"] = precision_score(y_true, y_pred, average="macro")
            results["recall"] = recall_score(y_true, y_pred, average="macro")
            results["f1"] = f1_score(y_true, y_pred, average="macro")
        else:  # Obliczanie wszystkich metryk, gdy calculate_proba=True
            for metric in self.metrics:
                #print(metric)
                if metric == "accuracy":
                    results["accuracy"] = accuracy_score(y_true, y_pred)
                elif metric == "precision":
                    results["precision"] = precision_score(y_true, y_pred, average="macro")
                elif metric == "recall":
                    results["recall"] = recall_score(y_true, y_pred, average="macro")
                elif metric == "f1":
                    results["f1"] = f1_score(y_true, y_pred, average="macro")
                elif metric == "roc_auc" and y_prob is not None:
                    print('Obliczam roc_auc')
                    results["roc_auc"] = self._calculate_roc_auc(y_true, y_prob)

        return results

    def _calculate_roc_auc(self, y_true, y_prob):
        """Oblicza AUC i rysuje krzywą ROC."""
        fpr, tpr, _ = roc_curve(y_true, y_prob, pos_label=1)
        roc_auc = auc(fpr, tpr)
        return roc_auc, fpr, tpr

    def _display_results(self, model, y_true, y_pred, results, verbose=True, calculate_proba=False):
        """Wyświetla metryki, macierz pomyłek i raport klasyfikacji."""
        if isinstance(model, Pipeline):
            model_name = model.steps[-1][1].__class__.__name__
        else:
            model_name = model.__class__.__name__

        # Tworzenie DataFrame'a z wynikami
        metrics_df = pd.DataFrame([results])

        if verbose:
            display(metrics_df)
            display(f"\nWyniki dla {model_name}:")

            # Wyświetlanie macierzy pomyłek i raportu klasyfikacji tylko gdy calculate_proba=True
            if calculate_proba:  
                fig, ax = plt.subplots(figsize=(8, 6))
                disp = ConfusionMatrixDisplay(
                    confusion_matrix=confusion_matrix(y_true, y_pred),
                    display_labels=self.display_labels
                    if self.display_labels
                    else np.unique(y_true),
                )
                disp.plot(ax=ax, cmap=plt.cm.Blues)
                plt.title("Macierz pomyłek")
                plt.show()

                report = classification_report(y_true, y_pred, output_dict=True)
                report_df = pd.DataFrame(report).transpose()
                display(report_df)

            if "roc_auc" in results:
                roc_auc, fpr, tpr = results["roc_auc"]
                self._plot_roc_curve(fpr, tpr, roc_auc)

        return metrics_df

    def _plot_roc_curve(self, fpr, tpr, roc_auc):
        """Rysuje krzywą ROC."""
        plt.figure()
        plt.plot(
            fpr,
            tpr,
            color="darkorange",
            lw=2,
            label=f"ROC curve (area = {roc_auc:.2f})",
        )
        plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curve")
        plt.legend(loc="lower right")
        plt.show()

### Sprawdzenie modelu wykorzystując VotingClassifier dla danych orginalnaych

In [ ]:
# Utworzenie estymatorów dla VotingClassifier
qda = QDA(reg_param=0.1)
hgb = HistGradientBoostingClassifier()
et = ExtraTreesClassifier()

# Utworzenie VotingClassifier
VotClass = VotingClassifier(
    estimators=[('qda', qda), ('hgb', hgb), ('et', et)],
    voting='soft' 
)
# Lista modeli do testowania
models = [VotClass]

# Tworzenie instancji klasy ModelEvaluator
evaluator = ModelEvaluator()

# Trenowanie i ewaluacja modeli
all_metrics = {}
for model in models:
    model_name = model.__class__.__name__

    # Utworzenie pipeline'u (możesz dodać kroki przetwarzania danych)
    pipeline = Pipeline([
        ("classifier", model)
    ])


    # Trenowanie i ewaluacja pipeline'u
    eval_results = evaluator.evaluate(
        
        pipeline, X_train_base, y_train_base, X_val_base, y_val_base, calculate_proba=True, verbose=True
    )
    all_metrics[model_name] = eval_results

# Tworzenie DataFrame z metrykami
metrics_df = pd.DataFrame(all_metrics, index=list(eval_results.keys()))

### Sprawdzenie modelu wykorzystując VotingClassifier dla danych upscalowanych

In [ ]:
# Utworzenie estymatorów dla VotingClassifier
qda = QDA(reg_param=0.1)
hgb = HistGradientBoostingClassifier()
et = ExtraTreesClassifier()

# Utworzenie VotingClassifier
VotClass = VotingClassifier(
    estimators=[('qda', qda), ('hgb', hgb), ('et', et)],
    voting='soft' 
)
# Lista modeli do testowania
models = [VotClass]

# Tworzenie instancji klasy ModelEvaluator
evaluator = ModelEvaluator()

# Trenowanie i ewaluacja modeli
all_metrics = {}
for model in models:
    model_name = model.__class__.__name__

    # Utworzenie pipeline'u (możesz dodać kroki przetwarzania danych)
    pipeline = Pipeline([("classifier", model)])

    # Trenowanie i ewaluacja pipeline'u
    eval_results = evaluator.evaluate(
        pipeline, X_train_Upsc, y_train_Upsc, X_val_Upsc, y_val_Upsc, calculate_proba=True, verbose=True
    )
    all_metrics[model_name] = eval_results

# Tworzenie DataFrame z metrykami
metrics_df = pd.DataFrame(all_metrics, index=list(eval_results.keys()))

# Wyświetlanie DataFrame z metrykami
# print("\nMetryki w formie tabeli:")
# display(metrics_df)

### Detekcja outlinerów oraz agumentacja danych - Testy Optuna

In [ ]:
from skimage.transform import AffineTransform, warp, ProjectiveTransform
from skimage.filters import gaussian, unsharp_mask, laplace
from skimage.morphology import erosion, closing, opening,  disk

warnings.filterwarnings("ignore", category=FutureWarning)
OPTUNA_module = optunahub.load_module(package="samplers/auto_sampler")


# X_train_Upsc, X_val_Upsc, y_train_Upsc, y_val_Upsc = train_test_split(
#     smoothed_X_train, y_train, test_size=0.2, random_state=69)


# Tworzenie instancji klasy ModelEvaluator
evaluator = ModelEvaluator()

def callback(study, trial):
    print(f"Trial {trial.number}:")
    print(f"  Wartość: {trial.value}")
    print(f"  Parametry: {trial.params}")

def create_voting_classifier():
    """Tworzy VotingClassifier z trzema estymatorami."""

    estimators = [
        ('qda', QDA(reg_param=0.1)),
        ('hgb', HistGradientBoostingClassifier()),
        ('et', ExtraTreesClassifier())
    ]
    return VotingClassifier(estimators=estimators, voting='soft')

def augment_data(X_train, y_train, n_neighbors_lof, contamination_lof, 
                    contamination_forest):
    """
    Augmentuje dane treningowe poprzez:
        1. Wykrywanie outlierów za pomocą LOF i IsolationForest.
        2. Augmentację danych dla outlierów (przesunięcie i obrót).
        3. Redukcję wymiarów UMAP dla całego zestawu danych.
        4. Wizualizację outlierów i ich augmentacji.
    """

    # 1. Redukcja wymiarów UMAP dla LOF
    reducer = umap.UMAP(n_components=2)
    X_train_reduced = reducer.fit_transform(X_train.reshape(X_train.shape[0], -1))

    # 2. Wykrywanie outlierów (LOF)
    lof = LocalOutlierFactor(n_neighbors=n_neighbors_lof, contamination=contamination_lof)
    y_pred_lof = lof.fit_predict(X_train_reduced)

    # 3. Isolation Forest na surowych danych
    iso_forest = IsolationForest(contamination=contamination_forest)
    y_pred_iso = iso_forest.fit_predict(X_train.reshape(X_train.shape[0], -1))

    # 4. Połączenie wyników LOF i IsolationForest
    y_pred_combined = (y_pred_lof + y_pred_iso) / 2
    outlier_indices = np.where(y_pred_combined < 0)[0]
    print(f"Liczba outlinerów: {len(outlier_indices)}")
    
    
    # 5. Augmentacja danych dla outlierów
    X_train_augmented = []
    y_train_augmented = []

    if len(outlier_indices) > 0:
        n_outliers_to_show = min(12, len(outlier_indices))  # Ograniczenie liczby wyświetlanych outlierów
        for i, idx in enumerate(outlier_indices[:n_outliers_to_show]):
            image = X_train[idx]
            label = y_train[idx]  # Pobranie etykiety dla outliera

            # 1. Dodanie paddingu (czarne tło)
            padded_image = np.pad(image, pad_width=35, mode='constant')  # Zwiększony padding
            for j in range(4):  # Zmienna j zdefiniowana wewnątrz pętli
                # 2. Transformacja afiniczna
                transform = AffineTransform(scale=(np.random.uniform(0.97, 1.04), np.random.uniform(0.97, 1.04)),
                                            rotation=np.deg2rad(np.random.randint(-20, 21)),
                                            shear=np.deg2rad(np.random.randint(-5, 6))
                                            )  
                transformed_image = warp(padded_image, transform, mode='wrap', preserve_range=True)
                
                # # 3. Wyizolowanie cyfry po transformacji 
                thresh = transformed_image > 20  
                contours = find_contours(thresh, 0.75)

                # --- Obsługa błędów ---
                try:
                    cnt = contours[0]  # Wybór pierwszego konturu

                    # Wyznaczenie bounding box
                    minr = int(np.min(cnt[:, 0]))
                    minc = int(np.min(cnt[:, 1]))
                    maxr = int(np.max(cnt[:, 0]))
                    maxc = int(np.max(cnt[:, 1]))

                    # Sprawdzenie, czy bounding box ma poprawny kształt
                    if minr == maxr or minc == maxc:
                        print("Nieprawidłowy bounding box - pomijanie augmentacji dla tego obrazu.")
                        continue

                except IndexError:
                    print("Nie znaleziono konturu - pomijanie augmentacji dla tego obrazu.")
                    continue

                # 4. Wycięcie cyfry na podstawie bounding box
                isolated_digit = transformed_image[minr:maxr, minc:maxc]
            
                # --- Erozja ---
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = erosion(isolated_digit, disk(1))
                    
                # --- Otwarcie  ---
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = opening(isolated_digit, disk(1))
                    
                # --- Zamknięcie 
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = closing(isolated_digit, disk(1))

                              
                
                # 5. Wycentrowanie cyfry w obrazie 42x42
                new_image = np.zeros((42, 42), dtype=np.uint8)
                row_offset = max(0, (42 - (maxr - minr)) // 2)
                col_offset = max(0, (42 - (maxc - minc)) // 2)
                new_image[row_offset:row_offset + maxr - minr, col_offset:col_offset + maxc - minc] = isolated_digit

                isolated_digit = new_image

                # 6. Dodanie do listy
                X_train_augmented.append(isolated_digit)  # Dodaj augmentowany obraz
                y_train_augmented.append(label)  # Dodaj etykietę

    X_train_augmented = np.array(X_train_augmented)
    y_train_augmented = np.array(y_train_augmented)

    # Połączenie danych augmentowanych z oryginalnymi danymi
    X_train_augmented = np.concatenate((X_train, X_train_augmented), axis=0)
    y_train_augmented = np.concatenate((y_train, y_train_augmented), axis=0)

    return X_train_augmented, y_train_augmented, outlier_indices, X_train_reduced
    
    
def objective(trial, X, y, use_umap):
    """Funkcja celu dla Optuna z podziałami train-test split dla każdego zestawu hiperparametrów."""

    # Optymalizacja hiperparametrów
    n_neighbors_lof = trial.suggest_int("n_neighbors_lof", 20, 800)
    contamination_lof = trial.suggest_float("contamination_lof", 0.005, 0.35, step=0.0025)
    contamination_forest = trial.suggest_float("contamination_forest", 0.005, 0.35, step=0.0025)

    if use_umap:
        n_components_umap = trial.suggest_int("n_components_umap", 2, 120)
        n_neighbors_UMAP = trial.suggest_int("n_neighbors", 2, 2400)
        min_dist = trial.suggest_uniform("min_dist", 0.0, 0.75)
        spread = trial.suggest_float("spread", 0.75, 2.0)
        local_connectivity = trial.suggest_float("local_connectivity", 1.0, 10.0)
        set_op_mix_ratio = trial.suggest_float("set_op_mix_ratio", 0.0, 1.0)

    scores = []
    for _ in range(3):
        # Podział danych na zbiór treningowy i walidacyjny
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
            
        # Augmentacja danych
        X_train_augmented, y_train_augmented, _, _ = augment_data(
            X_train, y_train, n_neighbors_lof, contamination_lof, contamination_forest
        )

        # Spłaszczenie danych przed ewaluacją
        X_train_augmented = X_train_augmented.reshape(X_train_augmented.shape[0], -1)
        X_val = X_val.reshape(X_val.shape[0], -1)

        # Ewaluacja modelu po augmentacji
        model = create_voting_classifier()

        if use_umap:
            pipeline = Pipeline([
                ("reducer", umap.UMAP(
                    n_components=n_components_umap,
                    n_neighbors=n_neighbors_UMAP,
                    set_op_mix_ratio=set_op_mix_ratio,
                    min_dist=min_dist,
                    spread=spread,
                    local_connectivity=local_connectivity
                )),
                ("classifier", model)
            ])
        else:
            pipeline = Pipeline([
                ("classifier", model)
            ])

        eval_results = evaluator.evaluate(
            pipeline, X_train_augmented, y_train_augmented, X_val, y_val,
            calculate_proba=False, verbose=False
        )

        # Zapisanie wyniku dla bieżącego podziału
        scores.append(sum([eval_results[metric] * 0.25 for metric in ["accuracy", "precision", "recall", "f1"]]))

    # Zwrócenie uśrednionego wyniku z trzech podziałów
    return np.mean(scores)


# --- Utworzenie storage ---
storage = optuna.storages.RDBStorage("sqlite:///study_Auto_sampler_New_1.db")

# --- Utworzenie lub wczytanie study ---
study_name_umap = "Test_1_umap"
study_name_no_umap = "Test_1_no_umap"

try:
    study_umap = optuna.load_study(study_name=study_name_umap, storage=storage)
    print("Wczytano study z UMAP z bazy danych.")
except KeyError:
    study_umap = optuna.create_study(direction="maximize", storage=storage, study_name=study_name_umap)
    print("Utworzono nowe study z UMAP.")

try:
    study_no_umap = optuna.load_study(study_name=study_name_no_umap, storage=storage)
    print("Wczytano study bez UMAP z bazy danych.")
except KeyError:
    study_no_umap = optuna.create_study(direction="maximize", storage=storage, study_name=study_name_no_umap, sampler=OPTUNA_module.AutoSampler())
    print("Utworzono nowe study bez UMAP.")

# Optymalizacja z UMAP
#study_umap.optimize(lambda trial: objective(trial, X_val_Upsc, y_val_Upsc, use_umap=True), n_trials=150,  n_jobs=4, gc_after_trial=True)

# Optymalizacja bez UMAP
study_no_umap.optimize(lambda trial: objective(trial, smoothed_X_train, y_train, use_umap=False), n_trials=250,  n_jobs=-1, gc_after_trial=True)

# Wyświetlenie najlepszych hiperparametrów dla agumentacji bez UMAP
print("Najlepsze parametry:", study_no_umap.best_params)
print("Najlepszy wynik:", study_no_umap.best_value)

# Wyświetlenie najlepszych hiperparametrów dla agumentacji z UMAP
# print("Najlepsze parametry:", study_umap.best_params)
# print("Najlepszy wynik:", study_umap.best_value)

# Baseline dla danych upscalowanyc bez redukcji wymiarów i agumentacji 0.9642205
######## study_Auto_sampler_New_1 <--- Obecna wersja operacji affiniacji i morfologcznmych. 
# Optymalizcja z UMAP ---->: 
# bez Umap                   [I 2025-01-07 13:31:21,997] Trial 237 finished with value: 0.9660396137534527 and parameters: {'n_neighbors_lof': 222, 'contamination_lof': 0.1625, 'contamination_forest': 0.195}. Best is trial 237 with value: 0.9660396137534527.


In [ ]:


# Tworzenie instancji klasy ModelEvaluator
evaluator = ModelEvaluator()

def create_voting_classifier():
    """Tworzy VotingClassifier z trzema estymatorami."""

    estimators = [
        ('qda', QDA(reg_param=0.1)),
        ('hgb', HistGradientBoostingClassifier()),
        ('et', ExtraTreesClassifier())
    ]
    return VotingClassifier(estimators=estimators, voting='soft')

def augment_data(X_train, y_train, n_neighbors_lof, contamination_lof, 
                    contamination_forest):
    """
    Augmentuje dane treningowe poprzez:
        1. Wykrywanie outlierów za pomocą LOF i IsolationForest.
        2. Augmentację danych dla outlierów (przesunięcie i obrót).
        3. Redukcję wymiarów UMAP dla całego zestawu danych.
        4. Wizualizację outlierów i ich augmentacji.
    """

    # 1. Redukcja wymiarów UMAP dla LOF
    reducer = umap.UMAP(n_components=2)
    X_train_reduced = reducer.fit_transform(X_train.reshape(X_train.shape[0], -1))

    # 2. Wykrywanie outlierów (LOF)
    lof = LocalOutlierFactor(n_neighbors=n_neighbors_lof, contamination=contamination_lof)
    y_pred_lof = lof.fit_predict(X_train_reduced)

    # 3. Isolation Forest na surowych danych
    iso_forest = IsolationForest(contamination=contamination_forest)
    y_pred_iso = iso_forest.fit_predict(X_train.reshape(X_train.shape[0], -1))

    # 4. Połączenie wyników LOF i IsolationForest
    y_pred_combined = (y_pred_lof + y_pred_iso) / 2
    outlier_indices = np.where(y_pred_combined < 0)[0]
    print(f"Liczba outlinerów: {len(outlier_indices)}")
    
    
    # 5. Augmentacja danych dla outlierów
    X_train_augmented = []
    y_train_augmented = []

    if len(outlier_indices) > 0:
        n_outliers_to_show = min(12, len(outlier_indices))  # Ograniczenie liczby wyświetlanych outlierów
        fig, axes = plt.subplots(n_outliers_to_show, 5, figsize=(15, 3 * n_outliers_to_show))
        for i, idx in enumerate(outlier_indices[:n_outliers_to_show]):
            image = X_train[idx]
            label = y_train[idx]  # Pobranie etykiety dla outliera
            # Oryginalny obraz
            axes[i, 0].imshow(image, cmap='gray')
            axes[i, 0].set_title(f'Outlier {i+1}')
            axes[i, 0].axis('off')

            # 1. Dodanie paddingu (czarne tło)
            padded_image = np.pad(image, pad_width=35, mode='constant')  # Zwiększony padding
            for j in range(4):  # Zmienna j zdefiniowana wewnątrz pętli
                # 2. Transformacja afiniczna
                transform = AffineTransform(scale=(np.random.uniform(0.97, 1.04), np.random.uniform(0.97, 1.04)),
                                            rotation=np.deg2rad(np.random.randint(-20, 21)),
                                            shear=np.deg2rad(np.random.randint(-5, 6))
                                            )  
                transformed_image = warp(padded_image, transform, mode='wrap', preserve_range=True)
                
                # # 3. Wyizolowanie cyfry po transformacji (wykrywanie konturów)
                thresh = transformed_image > 20  # Zwiększony próg
                contours = find_contours(thresh, 0.75)

                # --- Obsługa błędów ---
                try:
                    cnt = contours[0]  # Wybór pierwszego konturu

                    # Wyznaczenie bounding box
                    minr = int(np.min(cnt[:, 0]))
                    minc = int(np.min(cnt[:, 1]))
                    maxr = int(np.max(cnt[:, 0]))
                    maxc = int(np.max(cnt[:, 1]))

                    # Sprawdzenie, czy bounding box ma poprawny kształt
                    if minr == maxr or minc == maxc:
                        print("Nieprawidłowy bounding box - pomijanie augmentacji dla tego obrazu.")
                        continue

                except IndexError:
                    print("Nie znaleziono konturu - pomijanie augmentacji dla tego obrazu.")
                    continue

                # 4. Wycięcie cyfry na podstawie bounding box
                isolated_digit = transformed_image[minr:maxr, minc:maxc]
            
                # --- Erozja ---
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = erosion(isolated_digit, disk(1))
                    
                # --- Otwarcie  ---
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = opening(isolated_digit, disk(1))
                    
                # --- Zamknięcie 
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = closing(isolated_digit, disk(1))
                    
                if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                    isolated_digit = closing(isolated_digit, disk(1))
                              
                
                # 5. Wycentrowanie cyfry w obrazie 42x42
                new_image = np.zeros((42, 42), dtype=np.uint8)
                row_offset = max(0, (42 - (maxr - minr)) // 2)
                col_offset = max(0, (42 - (maxc - minc)) // 2)
                new_image[row_offset:row_offset + maxr - minr, col_offset:col_offset + maxc - minc] = isolated_digit

                isolated_digit = new_image

                # 6. Dodanie do listy
                X_train_augmented.append(isolated_digit)  # Dodaj augmentowany obraz
                y_train_augmented.append(label)  # Dodaj etykietę


                axes[i, j+1].imshow(new_image, cmap='gray')
                axes[i, j+1].axis('off')

        # plt.tight_layout()
        # plt.show()

    X_train_augmented = np.array(X_train_augmented)
    y_train_augmented = np.array(y_train_augmented)

    # Połączenie danych augmentowanych z oryginalnymi danymi
    X_train_augmented = np.concatenate((X_train, X_train_augmented), axis=0)
    y_train_augmented = np.concatenate((y_train, y_train_augmented), axis=0)

    return X_train_augmented, y_train_augmented, outlier_indices, X_train_reduced


# Zakładam, że masz zdefiniowane smoothed_X_train i y_train
X_train_Upsc, X_val_Upsc, y_train_Upsc, y_val_Upsc = train_test_split(
    smoothed_X_train, y_train, test_size=0.2, random_state=69)

# Wywołanie funkcji augment_data
X_train_augmented, y_train_augmented, outlier_indices, X_train_reduced = augment_data(
    X_train_Upsc,
    y_train_Upsc,
    n_neighbors_lof=399,
    contamination_lof=0.33,
    contamination_forest=0.005
)




In [ ]:
#Trial 34 finished with value: 0.9750009769150542 and parameters: {'n_neighbors_lof': 483, 'contamination_lof': 0.0475, 'contamination_forest': 0.10500000000000001, 'n_components_umap': 76, 'n_neighbors': 1436, 'min_dist': 0.13874920511220656, 'spread': 1.7486984209417518, 'local_connectivity': 2.5349453827107986, 'set_op_mix_ratio': 0.4209229951120764}. Best is trial 34 with value: 0.9750009769150542.
from tensorflow.keras.applications import EfficientNetV2B0, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L 
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


def create_efficientnetv2_model(
    input_shape,
    num_classes,
    efficientnet_version="B0",
    trainable=False,
    dropout_rate=0.2,
    weights=None
):
    """Tworzy model oparty na EfficientNetV2.

    Args:
        input_shape: Dane wejściowe.
        num_classes: Liczba klas.
        efficientnet_version: Wersja EfficientNetV2 ('B0', 'S', 'M', 'L').
        trainable: Czy warstwy EfficientNetV2 mają być trenowalne.
        dropout_rate: Współczynnik dropout.

    Returns:
        Model Keras.
    """

    # Wybór wersji EfficientNetV2
    if efficientnet_version == "B0":
        base_model = EfficientNetV2B0(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "S":
        base_model = EfficientNetV2S(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "M":
        base_model = EfficientNetV2M(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "L":
        base_model = EfficientNetV2L(
            include_top=False, weights=weights, input_shape=input_shape
        )
    else:
        raise ValueError("Nieprawidłowa wersja EfficientNetV2.")

    # Ustawienie warstw EfficientNetV2 jako trenowalne
    base_model.trainable = trainable

    # Budowa modelu
    inputs = Input(shape=input_shape)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

# Baseline data without transformations and agumentation 
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(smoothed_X_train, y_train_ENC, test_size=0.2, random_state=69)

# Odtworzenie augmentacji z najlepszymi parametrami
X_train_augmented, y_train_augmented = augment_data(
    X_train_base, 
    y_train_base, 
    n_neighbors_lof=399,
    contamination_lof=0.33,
    contamination_forest=0.005
)

X_train_base = np.expand_dims(X_train_base, axis=-1)
X_val_base = np.expand_dims(X_val_base, axis=-1)  
X_train_augmented = np.expand_dims(X_train_augmented, axis=-1)

# --- Trenowanie modeli ---
# Model na danych bez redukcji
model_base = create_efficientnetv2_model(
    input_shape=X_train_base.shape[1:], 
    num_classes=10, 
    efficientnet_version="B0",
    trainable=True,
    weights=None,
)

model_base.compile(
    optimizer="Lion", loss="categorical_crossentropy", metrics=["categorical_accuracy"] 
)

history_base = model_base.fit(
    X_train_base,
    y_train_base,
    epochs=16,
    batch_size=512,
    validation_data=(X_val_base, y_val_base),
)

# Model na danych z redukcją
model_transformed = create_efficientnetv2_model(
    input_shape=X_train_augmented.shape[1:],  
    num_classes=10, 
    efficientnet_version="B0",
    trainable=True,
    weights=None,
)

model_transformed.compile(
    optimizer="Lion", loss="categorical_crossentropy", metrics=["categorical_accuracy"] 
)

history_transformed = model_transformed.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=16,
    batch_size=512,
    validation_data=(X_val_base, y_val_base),
)

# --- Predykcja ---
y_pred_base = model_base.predict(X_val_base)
y_pred_transformed = model_transformed.predict(X_val_base)

# --- Porównanie wyników ---
def calculate_metrics(y_true, y_pred):
    """Oblicza metryki."""
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    return {
        "accuracy": accuracy_score(y_true_labels, y_pred_labels),
        "precision": precision_score(y_true_labels, y_pred_labels, average="macro"),
        "recall": recall_score(y_true_labels, y_pred_labels, average="macro"),
        "f1": f1_score(y_true_labels, y_pred_labels, average="macro"),
    }

metrics_base = calculate_metrics(y_val_base, y_pred_base)
metrics_transformed = calculate_metrics(y_val_base, y_pred_transformed)

print("Metryki dla modelu bez agumentacji:", metrics_base)
print("Metryki dla modelu z agumentacją:", metrics_transformed)



# Deklaracja funkcji dla ekstrakcji cech z obrazu

In [16]:

def calculate_hog_parallel(images, orientations=None, pixels_per_cell=None, cells_per_block=None, n_jobs=-1):
    """Oblicza deskryptory HOG dla obrazu."""
    
     # Ustawienie wartości domyślnych, jeśli parametry nie są podane
    if orientations is None:
        orientations = 9
    if pixels_per_cell is None:
        pixels_per_cell = (8, 8)
    if cells_per_block is None:
        cells_per_block = (2, 2)
    
    def calculate_hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block):
        
        image = img_as_float32(image)

        fd = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block, visualize=False, channel_axis=None)

        return fd
    
    return Parallel(n_jobs=n_jobs)(delayed(calculate_hog)(image, orientations, pixels_per_cell, cells_per_block) for image in images)


def apply_gabor_filters_parallel(images, thetas=None, sigmas=None, frequencies=None, n_jobs=-1):
    """
    Stosuje filtry Gabora do wielu obrazów równolegle.

    Argumenty:
        images: Lista obrazów wejściowych.
        thetas: (Opcjonalnie) Lista kątów orientacji filtrów Gabora (w radianach). Jeśli None, używane są wartości domyślne.
        sigmas: (Opcjonalnie) Lista odchyleń standardowych filtrów Gabora. Jeśli None, używane są wartości domyślne.
        frequencies: (Opcjonalnie) Lista częstotliwości przestrzennych filtrów Gabora. Jeśli None, używane są wartości domyślne.
        n_jobs: Liczba rdzeni procesora do wykorzystania podczas zrównoleglania. -1 oznacza użycie wszystkich dostępnych rdzeni.

    Zwraca:
        Listę tablic NumPy zawierających obrazy przefiltrowane filtrami Gabora, o typie danych float32.
    """

    # Ustawienie wartości domyślnych, jeśli parametry nie są podane
    if thetas is None:
        thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    if sigmas is None:
        sigmas = [1, 3]
    if frequencies is None:
        frequencies = [0.05, 0.25]

    def apply_gabor_filters_single(image):
        image = img_as_float32(image)
        filtered_images = [gabor(image, frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)[0]
                           for theta in thetas for sigma in sigmas for frequency in frequencies]
        return np.array(filtered_images, dtype=np.float32)

    return Parallel(n_jobs=n_jobs)(delayed(apply_gabor_filters_single)(image) for image in images)


def extract_features_HOG_Gabor(X, hog_orientations=None, hog_pixels_per_cell=None, hog_cells_per_block=None, 
                              gabor_thetas=None, gabor_sigmas=None, gabor_frequencies=None):
    """Generator zwracający cechy HOG i Gabora dla obrazów."""

    hog_features_list = calculate_hog_parallel(X, orientations=hog_orientations, pixels_per_cell=hog_pixels_per_cell, cells_per_block=hog_cells_per_block)
    gabor_features_list = apply_gabor_filters_parallel(X, thetas=gabor_thetas, sigmas=gabor_sigmas, frequencies=gabor_frequencies)

    for hog_features, gabor_features in zip(hog_features_list, gabor_features_list):
        
        gabor_features_flattened = np.array([img.flatten() for img in gabor_features])

        # Dopasowujemy kształt hog_features do gabor_features_flattened
        hog_features_reshaped = np.repeat(hog_features.reshape(1, -1), gabor_features_flattened.shape[0], axis=0)

        # Łączymy cechy HOG i Gabora w poziomie
        concatenated_features = np.hstack((hog_features_reshaped, gabor_features_flattened))

        concatenated_features = concatenated_features.flatten()

        yield concatenated_features

#################################################################################################################################################
################################################  Directional filters, ORB    ###################################################################

def apply_directional_filters(image, kernel_sizes=[3, 5]):
    """Stosuje filtry kierunkowe do obrazu."""
    filtered_images = []

    # Wykrywanie krawędzi filtrem Canny'ego
    edges = cv2.Canny(image, 50, 150)

    def process_kernel(kernel_size):
        if kernel_size % 2 == 0 or kernel_size > 31:
            raise ValueError(
                f"Nieprawidłowy rozmiar jądra: {kernel_size}. Rozmiar jądra musi być nieparzysty i nie większy niż 31."
            )
        kernel_horizontal = cv2.getDerivKernels(1, 0, kernel_size,
                                               normalize=True)
        kernel_vertical = cv2.getDerivKernels(0, 1, kernel_size,
                                             normalize=True)
        filtered = cv2.sepFilter2D(edges, cv2.CV_32F, kernel_horizontal[0],
                                  kernel_vertical[0])
        return filtered.flatten()

    with Pool() as pool:
        filtered_images = pool.map(process_kernel, kernel_sizes)

    # Analiza składowych spójnych
    labels = label(edges)
    props = regionprops(labels)
    features = [prop.area for prop in props]

    return np.concatenate(
        [np.array(filtered_images).squeeze(), np.array(features)])


def extract_orb_features(image, nfeatures=100):
    """Wyodrębnia cechy ORB z obrazu."""
    orb = cv2.ORB_create(nfeatures=nfeatures,
                         fastThreshold=12,
                         edgeThreshold=12)
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return []
    return descriptors.flatten()


def extract_combined_features(image, kernel_sizes=[3, 5], nfeatures=100):
    """Łączy cechy z filtrów kierunkowych i ORB."""
    directional_features = apply_directional_filters(image, kernel_sizes)
    orb_features = extract_orb_features(image, nfeatures)

    # Łączenie cech
    combined_features = np.concatenate((directional_features, orb_features))

    return combined_features
   
#################################################################################################################################################
################################################  GLCM, Zernike    ##############################################################################
def extract_glcm_features(image, distances=[1], angles=[0], properties=['contrast', 'energy', 'homogeneity', 'correlation']):
    """Wyodrębnia cechy GLCM z obrazu."""
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    glcm = graycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)

    features = []
    for prop in properties:
        features.extend(graycoprops(glcm, prop).flatten())

    return np.array(features)


## Testowanie modelu dla HOG i filtrów Gabora i porównanie wydajności przy zastosowaniu redukcji wymiarów. 

In [ ]:
def train_and_evaluate_models(X_train, y_train, use_pca=True):
    """
    Trenuje i ocenia modele QDA i SVC, opcjonalnie z redukcją wymiarów PCA.

    Argumenty:
        X_train: Dane treningowe.
        y_train: Etykiety treningowe.
        use_pca: Flaga wskazująca, czy używać PCA (domyślnie True).
    """

    # Ekstrakcja cech HOG i Gabor za pomocą generatora
    X_train_features = np.array(list(extract_features_HOG_Gabor(X_train)))

    # Podział na zbiór treningowy i walidacyjny
    X_train, X_val, y_train, y_val = train_test_split(X_train_features, y_train, test_size=0.2, random_state=69)

    # Redukcja wymiarów (opcjonalnie)
    if use_pca:
        #pca_dim = PCA(n_components=20)
        pca_dim = umap.UMAP(n_components=20, random_state=42)
        X_train = pca_dim.fit_transform(X_train)
        X_val = pca_dim.transform(X_val)

    # Tworzenie i trenowanie modelu QDA
    qda = QDA(reg_param=0.1)
    qda.fit(X_train, y_train)

    # Tworzenie i trenowanie modelu SVC
    svc = SVC(kernel='rbf')
    svc.fit(X_train, y_train)

    # Ocena modelu
    y_pred_qda = qda.predict(X_val)
    accuracy_qda = accuracy_score(y_val, y_pred_qda)

    y_pred_svc = svc.predict(X_val)
    accuracy_svc = accuracy_score(y_val, y_pred_svc)

    print("Dokładność QDA HOG i Gabor", "po PCA:" if use_pca else ":", accuracy_qda)
    print("Dokładność SVC HOG i Gabor", "po PCA:" if use_pca else ":", accuracy_svc)

# Test z PCA
train_and_evaluate_models(X_train, y_train, use_pca=True)
train_and_evaluate_models(X_train, y_train, use_pca=False)
#cProfile.run('train_and_evaluate_models(X_train, y_train, use_pca=True)') 

## OPTUNA dla HOG, Gabor, PCA

In [11]:
# import warnings
# warnings.filterwarnings("ignore", module="optuna")


# def objective(trial, X_train, y_train):
#     """
#     Funkcja celu dla optymalizacji Optuna. 
#     Przyjmuje próbę (trial), dane treningowe (X_train) i etykiety (y_train).
#     Zwraca dokładność QDA i SVM na danych walidacyjnych po zastosowaniu sekwencji transformacji.
#     """

#     # Parametry HOG
#     orientations      = trial.suggest_int("hog_orientations", 2, 16)
#     pixels_per_cell   = trial.suggest_categorical("hog_pixels_per_cell", [str((2, 2)), str((2, 3)), str((3, 2)), str((3, 3)), str((4, 4)), str((5, 5)),str((6, 6)), str((7, 7))])
#     cells_per_block   = trial.suggest_categorical("hog_cells_per_block", [str((1, 1)), str((2, 2)), str((3, 3)), str((4, 4))])

#     # Parametry filtrów Gabora
#     gabor_thetas      = trial.suggest_categorical("gabor_thetas", ([0],  [np.pi / 14],  [np.pi / 12], [0, np.pi / 14 ], [0, np.pi / 12 ] ))
#     gabor_sigmas      = trial.suggest_categorical("gabor_sigmas", ([0.33], [0.66], [1], [0.33 , 1.25], [0.66, 2.35], [1, 3]))
#     gabor_frequencies = trial.suggest_categorical("gabor_frequencies", ([0.025], [0.05], [0.125], [0.025, 0.125], [0.05, 0.175], [0.05, 0.25]))
        
#     #PCA
#     PCA_n_components  =  trial.suggest_int("PCA_Components", 4, 228)
#     kernel = trial.suggest_categorical("kernel_pca_kernel", ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])
#     gamma = None
#     if kernel in ['poly', 'rbf']:
#         gamma = trial.suggest_loguniform("kernel_pca_gamma", 0.001, 1)
        
#     # Powiększanie i wygładzanie
#     # X_train_split = np.array([enlarge_and_smooth(img, new_size) for img in X_train])
#     # X_val = np.array([enlarge_and_smooth(img, new_size) for img in X_val])
    
#     # Ekstrakcja cech HOG i Gabor
#     X_train_features = np.array(list(extract_features_HOG_Gabor(
#         X_train, 
#         hog_orientations=orientations, 
#         hog_pixels_per_cell=eval(pixels_per_cell), # Konwersja stringa na krotkę
#         hog_cells_per_block=eval(cells_per_block),  # Konwersja stringa na krotkę
#         gabor_thetas=gabor_thetas, 
#         gabor_sigmas=gabor_sigmas, 
#         gabor_frequencies=gabor_frequencies
#     )))

#     # Podział na zbiór treningowy i walidacyjny
#     X_train_filters, X_val_filters, y_train_filters, y_val_filters = train_test_split(X_train_features, y_train, test_size=0.2, random_state=69)
    
    
#     # Kernel PCA
#     kpca = KernelPCA(n_components=PCA_n_components, kernel=kernel, gamma=gamma)
#     X_train_reduced = kpca.fit_transform(X_train_filters)
#     X_val_reduced = kpca.transform(X_val_filters)
                
#     # SVC
#     svc = SVC(kernel='rbf')
#     svc.fit(X_train_reduced, y_train_filters)
#     y_pred_svm = svc.predict(X_val_reduced)
#     accuracy_svm = accuracy_score(y_val_filters, y_pred_svm)

#     return accuracy_svm

# # Optymalizacji
                            
# study = optuna.create_study(directions=["maximize"], sampler=optuna.samplers.CmaEsSampler() ) 
# study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, n_jobs=1)


# print("Best parameters:", study.best_params)
# print("Best QDA accuracy:", study.best_trial.values[0]) 
# print("Best SVM accuracy:", study.best_trial.values[1]) 

# # [I 2024-09-17 05:43:37,964] Trial 13 finished with value: 0.99133 and parameters: {'hog_orientations': 16, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(3, 3)', 'gabor_thetas': [0.2243994752564138], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.025], 'PCA_Components': 209, 'kernel_pca_kernel': 'cosine'}. Best is trial 13 with value: 0.9913333333333333.
# # [I 2024-09-17 08:45:58,345] Trial 19 finished with value: 0.99175 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(2, 3)', 'hog_cells_per_block': '(2, 2)', 'gabor_thetas': [0.2617993877991494], 'gabor_sigmas': [1], 'gabor_frequencies': [0.025], 'PCA_Components': 132, 'kernel_pca_kernel': 'sigmoid'}. Best is trial 19 with value: 0.99175.
# # [I 2024-09-17 10:45:29,605] Trial 23 finished with value: 0.99225 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(3, 3)', 'hog_cells_per_block': '(2, 2)', 'gabor_thetas': [0], 'gabor_sigmas': [1], 'gabor_frequencies': [0.05, 0.25], 'PCA_Components': 150, 'kernel_pca_kernel': 'cosine'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 13:24:59,082] Trial 28 finished with value: 0.99108 and parameters: {'hog_orientations': 9, 'hog_pixels_per_cell': '(3, 2)', 'hog_cells_per_block': '(1, 1)', 'gabor_thetas': [0, 0.2617993877991494], 'gabor_sigmas': [1], 'gabor_frequencies': [0.05, 0.25], 'PCA_Components': 190, 'kernel_pca_kernel': 'sigmoid'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 13:54:44,248] Trial 29 finished with value: 0.9915 and parameters: {'hog_orientations': 7, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0.2617993877991494], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.05], 'PCA_Components': 164, 'kernel_pca_kernel': 'cosine'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 18:02:03,444] Trial 37 finished with value: 0.99066 and parameters: {'hog_orientations': 9, 'hog_pixels_per_cell': '(3, 2)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.05, 0.175], 'PCA_Components': 169, 'kernel_pca_kernel': 'rbf', 'kernel_pca_gamma': 0.0032186977490984547}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 19:02:58,946] Trial 39 finished with value: 0.9915 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0, 0.2243994752564138], 'gabor_sigmas': [1, 3], 'gabor_frequencies': [0.125], 'PCA_Components': 148, 'kernel_pca_kernel': 'linear'}. Best is trial 23 with value: 0.99225.


## Wykorzystanie wcześniej otrzymanch hiperparamatrów dla HOG i filtrów Gabora po KPCA w celu stworzenia wektora cech z obrazów.

In [12]:
# X_train_features = np.array(list(extract_features_HOG_Gabor(
#         X_train, 
#         hog_orientations=6, 
#         hog_pixels_per_cell=eval('(3, 3)'), 
#         hog_cells_per_block=eval('(2, 2)'),  
#         gabor_thetas=[0], 
#         gabor_sigmas=[1], 
#         gabor_frequencies=[0.05, 0.25]
#     )))


# kpca = KernelPCA(n_components=150, kernel='cosine')

# X_train_filters, X_val_filters, y_train_filters, y_val_filters = train_test_split(X_train_features, y_train_ENC, test_size=0.2, random_state=69)

# X_train_reduced = kpca.fit_transform(X_train_filters)
# X_val_reduced = kpca.transform(X_val_filters)


# np.save('X_train_reduced.npy', X_train_reduced)
# np.save('X_val_reduced.npy', X_val_reduced)
# np.save('y_train_filters.npy', y_train_filters)
# np.save('y_val_filters.npy', y_val_filters)


### EfficientNetV2 

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L 
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

smoothed_X_train = np.array([enlarge_and_smooth(image, new_size=(42, 42)) for image in X_train])

# Baseline accuracy without transformations
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(
    smoothed_X_train, y_train_ENC, test_size=0.2, random_state=69)

X_train_base = np.expand_dims(X_train_base, axis=-1)  
X_val_base = np.expand_dims(X_val_base, axis=-1) 

print(X_train_base.shape)
print(X_val_base.shape)

def create_efficientnetv2_model(
    input_shape,
    num_classes,
    efficientnet_version="B0",
    trainable=False,
    dropout_rate=0.2,
    weights=None
):
    """Tworzy model oparty na EfficientNetV2.

    Args:
        input_shape: Dane wejściowe.
        num_classes: Liczba klas.
        efficientnet_version: Wersja EfficientNetV2 ('B0', 'S', 'M', 'L').
        trainable: Czy warstwy EfficientNetV2 mają być trenowalne.
        dropout_rate: Współczynnik dropout.

    Returns:
        Model Keras.
    """

    # Wybór wersji EfficientNetV2
    if efficientnet_version == "B0":
        base_model = EfficientNetV2B0(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "S":
        base_model = EfficientNetV2S(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "M":
        base_model = EfficientNetV2M(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "L":
        base_model = EfficientNetV2L(
            include_top=False, weights=weights, input_shape=input_shape
        )
    else:
        raise ValueError("Nieprawidłowa wersja EfficientNetV2.")

    # Ustawienie warstw EfficientNetV2 jako trenowalne
    base_model.trainable = trainable

    # Budowa modelu
    inputs = Input(shape=input_shape)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = create_efficientnetv2_model(
    input_shape=X_train_base.shape[1:],  # Przekazanie pełnego kształtu
    num_classes=10, 
    efficientnet_version="B0",
    trainable=True,
    weights=None,
)

model.summary()
model.compile(
        optimizer="Lion", loss="categorical_crossentropy", metrics=["categorical_accuracy"] 
)

history = model.fit(
    X_train_base,
    y_train_base,
    epochs=16,
    batch_size=512,
    validation_data=(X_val_base, y_val_base),
)

#### Pierwszy model na sieci konwolucyjnej dla bazy z HOG i Gabora.

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LayerNormalization , BatchNormalization, PReLU , Input, ActivityRegularization, GlobalAveragePooling1D, SeparableConv1D 
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Lion
from tensorflow.keras.callbacks import EarlyStopping
import gc



X_train_reduced = np.load('X_train_reduced.npy')
X_val_reduced = np.load('X_val_reduced.npy')
y_train_filters = np.load('y_train_filters.npy')
y_val_filters = np.load('y_val_filters.npy')

X_train_reduced = X_train_reduced.reshape(X_train_reduced.shape[0], X_train_reduced.shape[1], 1)
X_val_reduced = X_val_reduced.reshape(X_val_reduced.shape[0], X_val_reduced.shape[1], 1)

def calculate_max_pool_size(input_length, prev_pool_size, min_output_size=2):
    calculated_pool_size = input_length // prev_pool_size
    return max(min_output_size, calculated_pool_size)


def objective(trial):
    """Funkcja celu dla Optuny."""
    #os.environ['TF_NUM_INTRAOP_THREADS'] = '1' ## Czy Kers działa na pojedycznym wątku ????
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()

    activation_functions = ['relu', 'elu', 'leaky_relu', 'tanh']
    activation = trial.suggest_categorical('activation', activation_functions)

    # Definiowanie modelu
    model = Sequential()
    model.add(Input(shape=(X_train_reduced.shape[1], 1)))

    # Pierwsza warstwa konwolucyjna
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_1', 48, 450),
                     kernel_size=trial.suggest_int('kernel_size_1', 2, 16),
                     #activation=activation,
                     padding='same', 
                     #dilation_rate=trial.suggest_int('dilation_rate_1', 1, 4)
                     )
              )
    model.add(PReLU())                #$$$$$$$ LAYER przed PRelu
    model.add(LayerNormalization())   ######
    model.add(ActivityRegularization(l1=trial.suggest_float('l1_1', 1e-6, 1e-1, log=True), 
                                     l2=trial.suggest_float('l2_1', 1e-6, 1e-1, log=True)
                                     )
              )
    pool_size_1 = trial.suggest_int('pool_size_1', 2, 10)
    model.add(MaxPooling1D(pool_size=pool_size_1))



    # Druga warstwa konwolucyjna
    input_length_2 = X_train_reduced.shape[1] // pool_size_1
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_2', 24, 400),
                     kernel_size=trial.suggest_int('kernel_size_2', 2, 14),
                     padding='same',
                     #activation=activation,
                     #dilation_rate=trial.suggest_int('dilation_rate_2', 1, 4)
                     )
              )
    model.add(PReLU())
    model.add(LayerNormalization())
    model.add(ActivityRegularization(l1=trial.suggest_float('l1_2', 1e-6, 1e-1, log=True), 
                                     l2=trial.suggest_float('l2_2', 1e-6, 1e-1, log=True)
                                     )
              )
    pool_size_2 = trial.suggest_int('pool_size_2', 2, input_length_2 // 2)
    #pool_size_2 = trial.suggest_int('pool_size_2', 2, 10)
    model.add(MaxPooling1D(pool_size=pool_size_2))



    # Trzecia warstwa konwolucyjna
    #input_length_3 = input_length_2 // pool_size_2
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_3', 32, 300),
                     kernel_size=trial.suggest_int('kernel_size_3', 2, 12),
                     padding='same',
                     )
              )
    model.add(PReLU())
    model.add(LayerNormalization())
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_3', 0.01, 0.5)
                      )
              )
    #pool_size_3 = trial.suggest_int('pool_size_3', 2, max(2, input_length_3 // 2))
    #pool_size_3 = trial.suggest_int('pool_size_3', 2, 8)
    #model.add(MaxPooling1D(pool_size=pool_size_3))

    # Warstwy Dense
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(units=trial.suggest_int('dense_units_1', 40, 300), activation=activation))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_4', 0.01, 0.75)))
    
    model.add(Dense(units=trial.suggest_int('dense_units_2', 20, 200), activation=activation))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_5', 0.01, 0.65)))
    
    model.add(Dense(10, activation='softmax'))

    # Optymalizator
    lr_initial = trial.suggest_float("lr_initial", 1e-5, 1e-1, log=True)
    decay_steps = trial.suggest_int("decay_steps", 10, 150)
    decay_rate = trial.suggest_float("decay_rate", 0.01, 0.99)
    lr_schedule = ExponentialDecay(initial_learning_rate=lr_initial,
                                  decay_steps=decay_steps,
                                  decay_rate=decay_rate)

    optimizer_name = trial.suggest_categorical('optimizer', ['Lion'])
    optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    optimizer = optimizer_class(learning_rate=lr_schedule)

    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=16, start_from_epoch=10, min_delta = 0.00005)

    # Trenowanie modelu
    model.fit(X_train_reduced, y_train_filters,
              epochs=trial.suggest_int('epochs', 40, 150),
              batch_size=trial.suggest_int('batch_size', 64, 1920),
              validation_data=(X_val_reduced, y_val_filters),
              verbose=1,
              callbacks=[early_stopping])

    # Ewaluacja modelu
    _, accuracy = model.evaluate(X_val_reduced, y_val_filters, verbose=0)

    del model
    return accuracy

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) ## Czy keras działa na wszystkich wątkach, ale optuna na pojedynczym. 


# Najlepsze parametry i wynik
print("Najlepsze parametry:", study.best_params)
print("Najlepsza dokładność:", study.best_value)

#[I 2024-10-04 05:18:28,377] Trial 26 finished with value: 0.9810000061988831 and parameters: {'activation': 'elu', 'conv1d_filters_1': 347, 'kernel_size_1': 7,  'l1_1': 0.0002141584459916536, 'l2_1': 4.201061472934882e-06, 'pool_size_1': 6, 'conv1d_filters_2': 77, 'kernel_size_2': 14, 'l1_2': 3.203529032832527e-06, 'l2_2': 0.00015793896479891322, 'pool_size_2': 2, 'conv1d_filters_3': 258, 'kernel_size_3': 4, 'dropout_rate_3': 0.3297488074504043, 'dense_units_1': 271, 'dropout_rate_4': 0.5097723961373417, 'dense_units_2': 78, 'dropout_rate_5': 0.15473996425601394, 'lr_initial': 0.0004175922071628326, 'decay_steps': 33, 'decay_rate': 0.9780884395210858, 'optimizer': 'Lion', 'epochs': 123, 'batch_size': 264}. Best is trial 26 with value: 0.9810000061988831.
#[I 2024-10-04 14:58:14,236] Trial 61 finished with value: 0.9797499775886536 and parameters: {'activation': 'elu', 'conv1d_filters_1': 424, 'kernel_size_1': 13, 'l1_1': 0.0005501543495144062, 'l2_1': 0.02794377888054429,   'pool_size_1': 7, 'conv1d_filters_2': 79, 'kernel_size_2': 12, 'l1_2': 1.3699264627909325e-06, 'l2_2': 6.577814668979489e-06, 'pool_size_2': 3, 'conv1d_filters_3': 280, 'kernel_size_3': 3, 'dropout_rate_3': 0.21341586312675653, 'dense_units_1': 292, 'dropout_rate_4': 0.22415600475665626, 'dense_units_2': 106, 'dropout_rate_5': 0.09686035013603865, 'lr_initial': 0.0004430745906851504, 'decay_steps': 22, 'decay_rate': 0.9872365423551039, 'optimizer': 'Lion', 'epochs': 150, 'batch_size': 216}. Best is trial 26 with value: 0.9810000061988831.


## Sieć CONV2D z zamrożonym PCA dla danych pierwotnych

In [14]:
# import tensorflow as tf
# import keras
# from keras import layers
# import tensorflow_probability as tfp
# import optuna
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
# X_train_image = X_train_image.astype('float32')
# X_val_image = X_val_image.astype('float32')
# X_train_image = np.expand_dims(X_train_image, axis=-1)  
# X_val_image = np.expand_dims(X_val_image, axis=-1) 
 
# def objective(trial):
#     # Przestrzeń hiperparametrów
#     num_filters1 = trial.suggest_int("num_filters1", 32, 256)    # Pierwsza warstwa Conv2d
#     num_filters2 = trial.suggest_int("num_filters2", 64, 256)    # Druga warstwa Conv2d
#     num_filters3 = trial.suggest_int("num_filters3", 128, 256)   # Trzecia warstwa Conv2d
#     n_components = trial.suggest_int("PCA_components", 350, 500) # PCA

#     l1_1=trial.suggest_float('l1_1', 1e-6, 1e-1, log=True)
#     l2_1=trial.suggest_float('l2_1', 1e-6, 1e-1, log=True)
    
#     l1_2=trial.suggest_float('l1_2', 1e-6, 1e-1, log=True)
#     l2_2=trial.suggest_float('l2_2', 1e-6, 1e-1, log=True)

#     l1_3=trial.suggest_float('l1_3', 1e-6, 1e-1, log=True)
#     l2_3=trial.suggest_float('l2_3', 1e-6, 1e-1, log=True)
    
#     activation_functions = ['relu', 'gelu', 'leaky_relu', 'silu', 'mish']
#     activation = trial.suggest_categorical('activation', activation_functions)
    
    
#     # Warstwy konwolucyjne 2D
#     conv_layers = [
#         # Pierwsza warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters1,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_1),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_1),
#                                 padding='same',
#                                 input_shape=(X_train_image.shape[1:])
#                                 ),
        
#         tf.keras.layers.MaxPool2D(pool_size=2,strides=1,
#                                  ),
#         # Druga warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters2,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_2),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_2),                                
#                                 padding='same',
#                                 ),
#         tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
#         # Trzecia warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters3,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_3),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_3),                                
#                                 padding='same'
#                                 ),
#         tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
#         tf.keras.layers.Flatten(),
#                 ]

#     ### Obliczenia dla zamrożonego PCA na podstawie pierwszej inicjalizacji modelu
#     conv_model = keras.models.Sequential(conv_layers)  
#     X_train_PCA = conv_model.predict(X_train_image)
    
#     X_train_PCA = tf.reshape(X_train_PCA, [X_train_PCA.shape[0], -1])

#     # Macierz kowariancji
#     covariance_matrix = tfp.stats.covariance(X_train_PCA, sample_axis=0, event_axis=-1)
#     # Wektory i wartości własne
#     eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
#     sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
#     eigenvalues = tf.gather(eigenvalues, sorted_indices)
#     eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
    
#     #### Opcja druga na SVD, ale wymaga jeszcze zabawy
#     # s, u, v = tf.linalg.svd(covariance_matrix)
#     # eigenvectors = v
#     # eigenvalues = tf.square(s) 
    
#     # Warstwa PCA
#     model = keras.models.Sequential(conv_layers + [
#         tf.keras.layers.Lambda(lambda x, eigenvectors: tf.matmul(x, eigenvectors), arguments={'eigenvectors': eigenvectors[:, :n_components]})
#     ])

#     # Output (klasyfikacja)
#     model.add(tf.keras.layers.Dense(10, activation='softmax'))
#     #model.summary()
#     tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    

#     # Optymalizator
#     optimizer_name = trial.suggest_categorical('optimizer', ['AdamW'])
#     optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
#     # Początkową wartość learning rate
#     lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
#     optimizer = optimizer_class(learning_rate=lr_initial)  

#     ## Kompilacja modelu
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     # Early Stopping
#     early_stopping = EarlyStopping(monitor='categorical_accuracy', patience=8, start_from_epoch=8, min_delta=0.0001)
    
#     # ReduceLROnPlateau
#     reduce_lr = ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.9, step=0.1),
#         patience=trial.suggest_int("reduce_lr_patience", 2, 12),
#         min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-4, log=True)
#     )
    
#     # Trenowanie i ocena modelu
#     history = model.fit(X_train_image, y_train_image,
#                         epochs=trial.suggest_int('epochs', 15, 80),
#                         batch_size=trial.suggest_int('batch_size', 64, 1920),
#                         validation_data=(X_val_image, y_val_image),
#                         callbacks=[early_stopping, reduce_lr]
#                         )
    
#     accuracy = history.history['val_categorical_accuracy'][-1]

#     return accuracy


# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) 

# # Najlepsze parametry i wynik
# print("Najlepsze parametry:", study.best_params)
# print("Najlepsza dokładność:", study.best_value)

# ### dwa tygodnie więc lepiej aby te wartości stanowiły dobrą podstawę...
# #Trial 76 finished with value:  0.9906666874885559 and parameters: {'num_filters1': 194, 'num_filters2': 167, 'num_filters3': 221, 'PCA_components': 383, 'l1_1': 1.4917571854023011e-06, 'l2_1': 0.0003844462705458024,  'l1_2': 0.0011921068803372,     'l2_2': 0.00021948428596782946, 'l1_3': 0.05677951548917238,    'l2_3': 7.714986268695782e-05,  'activation': 'relu', 'optimizer': 'AdamW', 'lr_initial': 0.006869840110235172, 'reduce_lr_factor': 0.2, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.028747844465778e-06,  'epochs': 51, 'batch_size': 479}.
# #Trial 99 finished with value:  0.9911666512489319 and parameters: {'num_filters1': 222, 'num_filters2': 220, 'num_filters3': 202, 'PCA_components': 367, 'l1_1': 4.875844948451946e-06,  'l2_1': 0.00014963179483695857, 'l1_2': 2.3622464527905993e-06, 'l2_2': 0.0007506481920246097,  'l1_3': 0.030678868126237845,   'l2_3': 0.00016514043221099142, 'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.00653453290497133,  'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.8941869280860333e-06, 'epochs': 54, 'batch_size': 331}.
# #Trial 108 finished with value: 0.9915000200271606 and parameters: {'num_filters1': 205, 'num_filters2': 206, 'num_filters3': 220, 'PCA_components': 370, 'l1_1': 9.95600828812184e-06,   'l2_1': 4.680137051239973e-05,  'l1_2': 4.9076000996726795e-06, 'l2_2': 8.638060092445147e-05,  'l1_3': 0.0361709248993415,     'l2_3': 0.0007397174884371887,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006269166650314777, 'reduce_lr_factor': 0.2, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 5.203896150606987e-06,  'epochs': 62, 'batch_size': 239}.
# #Trial 121 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 189, 'num_filters2': 200, 'num_filters3': 220, 'PCA_components': 387, 'l1_1': 8.32443244291399e-05,   'l2_1': 5.963850113018811e-05,  'l1_2': 4.8017415472934674e-06, 'l2_2': 3.9423405128005956e-05, 'l1_3': 0.00047971791859799064, 'l2_3': 0.0014900662405900767,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006533479568732247, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 6, 'reduce_lr_min_lr': 7.315164005922789e-06,  'epochs': 58, 'batch_size': 249}.
# #Trial 134 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 180, 'num_filters2': 196, 'num_filters3': 234, 'PCA_components': 390, 'l1_1': 4.148854564296224e-05,  'l2_1': 2.3209137723337733e-05, 'l1_2': 0.0009209306936723504,  'l2_2': 1.9816087702628798e-05, 'l1_3': 0.0008993453039409338,  'l2_3': 0.002218879345590759,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007404316715937198, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 3.862546224445297e-07,  'epochs': 61, 'batch_size': 368}.
# #Trial 151 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 201, 'num_filters2': 189, 'num_filters3': 229, 'PCA_components': 403, 'l1_1': 6.15892270118e-05,      'l2_1': 1.6056284141459273e-05, 'l1_2': 0.0009167391639733724,  'l2_2': 1.4409315946656477e-05, 'l1_3': 0.0008899963895277467,  'l2_3': 0.005080838638011425,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007378956521980373, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 1.9759552872681682e-07, 'epochs': 63, 'batch_size': 416}.
# #Trial 153 finished with value: 0.9905833601951599 and parameters: {'num_filters1': 158, 'num_filters2': 190, 'num_filters3': 229, 'PCA_components': 405, 'l1_1': 3.317865892202326e-05,  'l2_1': 1.715760099034611e-05,  'l1_2': 0.00028475050803045533, 'l2_2': 7.144046413508896e-06,  'l1_3': 0.0010057548651594868,  'l2_3': 0.001544820248650564,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007667044780464368, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 1.6284942102179344e-07, 'epochs': 64, 'batch_size': 420}.
# #Trial 189 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 187, 'num_filters2': 194, 'num_filters3': 191, 'PCA_components': 381, 'l1_1': 0.0001222508587314991,  'l2_1': 5.2900939483419044e-05, 'l1_2': 4.050810455502403e-05,  'l2_2': 0.00017281194315162931, 'l1_3': 0.0014341620209873955,  'l2_3': 0.0025511470455798512,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006996548564114207, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 6, 'reduce_lr_min_lr': 9.663271049317225e-07,  'epochs': 60, 'batch_size': 219}.
# #Trial 205 finished with value: 0.9904999732971191 and parameters: {'num_filters1': 210, 'num_filters2': 189, 'num_filters3': 198, 'PCA_components': 483, 'l1_1': 8.253178154361272e-05,  'l2_1': 1.7866032331171518e-05, 'l1_2': 0.0008019764970918962,  'l2_2': 1.0316537375038554e-05, 'l1_3': 0.000838618983241391,   'l2_3': 0.008249547665798275,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006570979651986537, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.6114120319881016e-07, 'epochs': 63, 'batch_size': 560}.


#### V2 dynamic PCA

In [15]:
# import tensorflow as tf
# import keras
# from keras import layers
# import tensorflow_probability as tfp
# import optuna
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 

# X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
# X_train_image = X_train_image.astype('float32')
# X_val_image = X_val_image.astype('float32')
# X_train_image = np.expand_dims(X_train_image, axis=-1)  
# X_val_image = np.expand_dims(X_val_image, axis=-1) 
 
# def objective(trial):
#     # Przestrzeń hiperparametrów
#     num_filters1 = trial.suggest_int("num_filters1", 180, 230)    # Pierwsza warstwa Conv2d
#     num_filters2 = trial.suggest_int("num_filters2", 180, 230)    # Druga warstwa Conv2d
#     num_filters3 = trial.suggest_int("num_filters3", 180, 260)    # Trzecia warstwa Conv2d
#     n_components = trial.suggest_int("PCA_components", 350, 420)  # PCA
    
#     l1_1=trial.suggest_float('l1_1', 1e-6, 5e-3, log=True)
#     l2_1=trial.suggest_float('l2_1', 5e-5, 5e-3, log=True)
    
#     l1_2=trial.suggest_float('l1_2', 1e-6, 1e-3, log=True)
#     l2_2=trial.suggest_float('l2_2', 1e-6, 1e-5, log=True)

#     l1_3=trial.suggest_float('l1_3', 1e-6, 1e-1, log=True)
#     l2_3=trial.suggest_float('l2_3', 1e-6, 1e-1, log=True)
    
#     activation_functions = ['gelu', 'leaky_relu', 'silu', 'mish']
#     activation = trial.suggest_categorical('activation', activation_functions)
    
    
#     class DynamicPCA(keras.layers.Layer):
#         def __init__(self, n_components, layer, update_freq=2, threshold=0.5, **kwargs):  
#             super(DynamicPCA, self).__init__(**kwargs)
#             self.n_components = n_components
#             self.threshold = threshold
#             self.layer = layer  
#             self.update_freq = update_freq
#             self.val_loss = None
#             self.eigenvectors = None
#             self.epoch_count = 0

#         def build(self, input_shape):
#             self.eigenvectors = self.add_weight(
#                 shape=(input_shape[-1], self.n_components),
#                 initializer="zeros",
#                 trainable=True,
#             )

#         def call(self, inputs):
#             self.epoch_count += 1
#             if (self.val_loss is not None and 
#                 self.val_loss < self.threshold and 
#                 self.epoch_count % self.update_freq == 0):
#                 print("DynamicPCA: Aktualizacja wektorów własnych!")
#                 print(f"val_loss: {self.val_loss}, threshold: {self.threshold}, epoch_count: {self.epoch_count}, update_freq: {self.update_freq}")
                
#                 # Pobierz wagi bezpośrednio z warstwy
#                 weights = self.layer.get_weights()  
#                 flat_weights = tf.concat([tf.reshape(w, [-1]) for w in weights], axis=0)
#                 # Normalizacja wag
#                 flat_weights = tf.math.l2_normalize(flat_weights)
#                 # Obliczanie nowych wektorów własnych
#                 new_eigenvectors = self.calculate_pca(flat_weights)
#                 # Aktualizacja wag (wektorów własnych)
#                 self.eigenvectors.assign(new_eigenvectors)

#             # Mnożenie przez wektory własne
#             output = tf.matmul(inputs, self.eigenvectors)
#             return output

#         def calculate_pca(self, inputs):
#             print(flat_weights.shape)
#             # Obliczanie macierzy kowariancji
#             covariance_matrix = tf.linalg.cov(inputs)  
#             # Wektory i wartości własne
#             eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
#             sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
#             eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
#             return eigenvectors[:, :self.n_components]

#     class MyCallback(keras.callbacks.Callback):
#         def on_epoch_end(self, epoch, logs=None):
#             pca_layer = self.model.get_layer('dynamic_pca')
#             pca_layer.val_loss = logs.get('val_loss')
#             print(logs.get('val_loss'))

#     # Definiowanie wejścia
#     image_input = keras.Input(shape=X_train_image.shape[1:])  

#     # Warstwy konwolucyjne
#     conv1 = keras.layers.Conv2D(num_filters1, kernel_size=(2, 2), activation=activation)(image_input)
#     pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = keras.layers.Conv2D(num_filters2, kernel_size=(2, 2), activation=activation)(pool1)
#     pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = keras.layers.Conv2D(num_filters3, kernel_size=(2, 2), activation=activation)(pool2)
#     pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
#     flat_cnn = keras.layers.Flatten()(pool3)

#     # Dense 1
#     dense = keras.layers.Dense(units=trial.suggest_int('dense_units_1', 60, 240), activation=activation)(flat_cnn) 
#     dense = keras.layers.Dropout(rate=trial.suggest_float('dropout_rate_1', 0.01, 0.75))(dense)

#     # Warstwa DynamicPCA 
#     pca_output = DynamicPCA(n_components=n_components, layer=conv3, name='dynamic_pca', threshold=1.25)(flat_cnn)  
    
#     merged = keras.layers.Concatenate()([dense, pca_output])

#     dense2 = keras.layers.Dense(units=trial.suggest_int('dense_units_2', 60, 240), activation=activation)(merged)  
#     dense2 = keras.layers.Dropout(rate=trial.suggest_float('dropout_rate_2', 0.01, 0.75))(dense2)

#     # Output (klasyfikacja)
#     outputs = keras.layers.Dense(10, activation='softmax')(dense2)

#     # Utworzenie modelu
#     model = keras.Model(inputs=image_input, outputs=outputs)

#     # Utworzenie modelu
#     model = keras.Model(inputs=image_input, outputs=outputs)
#     #model.summary()
#     tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    

#     # Optymalizator
#     optimizer_name = trial.suggest_categorical('optimizer', ['AdamW'])
#     optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
#     # Początkową wartość learning rate
#     lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
#     optimizer = optimizer_class(learning_rate=lr_initial)  

#     ## Kompilacja modelu
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     # Early Stopping
#     early_stopping = EarlyStopping(monitor='categorical_accuracy', patience=10, start_from_epoch=8, min_delta=0.0001)
    
#     # ReduceLROnPlateau
#     reduce_lr = ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.5, step=0.1),
#         patience=trial.suggest_int("reduce_lr_patience", 4, 8),
#         min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-5, log=True)
#     )
    
#     # Trenowanie i ocena modelu
#     history = model.fit(X_train_image, y_train_image,
#                         epochs=trial.suggest_int('epochs', 20, 80),
#                         batch_size=trial.suggest_int('batch_size', 164, 768),
#                         validation_data=(X_val_image, y_val_image),
#                         validation_freq=1,
#                         callbacks=[early_stopping, reduce_lr, MyCallback()]
#                         )
    
#     accuracy = history.history['val_categorical_accuracy'][-1]

#     return accuracy


# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) 

# # Najlepsze parametry i wynik
# print("Najlepsze parametry:", study.best_params)
# print("Najlepsza dokładność:", study.best_value)



## Model bazujący na dwóch potokach danych. Surowych przechodzących przez Conv2d, oraz LSTM dla danych z HOG i Gabor. Wykorzystanie mechanizmu atencji. 

In [ ]:
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import Constraint

from tensorflow.keras.layers import ( Input, LSTM, GlobalAveragePooling1D, Permute, PReLU, Flatten, Reshape, Conv2D, MaxPooling2D, Concatenate, Attention, Dense,
                                     LayerNormalization, ActivityRegularization, Dropout, GlobalAveragePooling2D, AveragePooling2D, AveragePooling1D, GlobalAveragePooling1D, 
                                     GlobalMaxPool1D, GlobalMaxPool2D, MaxPool2D, SeparableConv1D, RandomRotation, BatchNormalization
                                    )
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gc
from tensorflow.keras.callbacks import TensorBoard

X_train_reduced = np.load('X_train_reduced.npy')
X_val_reduced = np.load('X_val_reduced.npy')
y_train_filters = np.load('y_train_filters.npy')
y_val_filters = np.load('y_val_filters.npy')

X_train_reduced = X_train_reduced.reshape(X_train_reduced.shape[0], X_train_reduced.shape[1], 1)
X_val_reduced = X_val_reduced.reshape(X_val_reduced.shape[0], X_val_reduced.shape[1], 1)

X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
X_train_image = X_train_image.astype('float32')
X_val_image = X_val_image.astype('float32')
X_train_image = np.expand_dims(X_train_image, axis=-1)  
X_val_image = np.expand_dims(X_val_image, axis=-1) 

class SpectralNorm(Constraint):
    """
    Spectral Normalization constraint.

    Args:
        iteration: Number of iterations for power iteration.
    """
    def __init__(self, iteration=1):
        super(SpectralNorm, self).__init__()
        self.iteration = iteration
        self.u = None

    def __call__(self, w):
        w_shape = w.shape.as_list()
        w = tf.reshape(w, [-1, w_shape[-1]])

        if self.u is None:
            self.u = tf.Variable(
                initial_value=tf.keras.initializers.RandomNormal()(shape=[1, w_shape[-1]]),
                trainable=False
            )

        if self.iteration == 1:
            v_hat = tf.einsum('ij,kj->ik', self.u, w) 
            v_hat = v_hat / tf.linalg.norm(v_hat, ord=2, axis=-1, keepdims=True)
            u_hat = tf.einsum('ij,jk->ik', v_hat, w)
            u_hat = u_hat / tf.linalg.norm(u_hat, ord=2, axis=-1, keepdims=True)
        else:
            def body(i, u_hat):
                v_hat = tf.einsum('ij,kj->ik', u_hat, w)
                v_hat = v_hat / tf.linalg.norm(v_hat, ord=2, axis=-1, keepdims=True)
                u_hat = tf.einsum('ij,jk->ik', v_hat, w)
                u_hat = u_hat / tf.linalg.norm(u_hat, ord=2, axis=-1, keepdims=True)
                return i + 1, u_hat

            _, u_hat = tf.while_loop(
                lambda i, _: i < self.iteration,
                body,
                loop_vars=[0, self.u]
            )

        self.u.assign(u_hat)  # Aktualizacja self.u po pętli

        w_norm = w / tf.linalg.norm(w, ord=2, axis=0, keepdims=True)
        w_norm = tf.reshape(w_norm, w_shape)

        return w_norm
    
def create_sepconv1d_layer(units, activation, l1, l2):
    return SeparableConv1D(
        units,
        kernel_size=1,
        padding='same',
        activation=activation,
        depthwise_constraint=SpectralNorm,
        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    )

def create_conv2d_layer(filters, activation, l1, l2):
    return Conv2D(
        filters,
        kernel_size=(2, 2),
        activation=activation,
        kernel_constraint=SpectralNorm,
        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    )    
    
def objective(trial):
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    # Dane wejściowe
    image_input = Input(shape=X_train_image.shape[1:])
    flattened_input = Input(shape=X_train_reduced.shape[1:])

    ### Hiperparamatry 
    activation_functions = ['gelu', 'leaky_relu', 'silu', 'mish']
    activation = trial.suggest_categorical('activation', activation_functions)
        
    filters1_1D=trial.suggest_int('Sep-Conv2d filters1', 32, 384)  # Pierwsza warstwa SeparableConv1D
    filters2_1D=trial.suggest_int('Sep-Conv2d filters2', 32, 384)  # Druga warstwa SeparableConv1D
    filters3_1D=trial.suggest_int('Sep-Conv2d filters3', 32, 384)  # Trzecia warstwa SeparableConv1D
    
    l1_1_1D=trial.suggest_float('l1_1_1D', 1e-6, 1e-1, log=True)
    l2_1_1D=trial.suggest_float('l2_1_1D', 1e-6, 1e-1, log=True)
    l1_2_1D=trial.suggest_float('l1_2_1D', 1e-6, 1e-1, log=True)
    l2_2_1D=trial.suggest_float('l2_2_1D', 1e-6, 1e-1, log=True)
    l1_3_1D=trial.suggest_float('l1_3_1D', 1e-6, 1e-1, log=True)
    l2_3_1D=trial.suggest_float('l2_3_1D', 1e-6, 1e-1, log=True)
       
    filters1_2D = trial.suggest_int("Conv2d filters1", 32, 384)    # Pierwsza warstwa Conv2d
    filters2_2D = trial.suggest_int("Conv2d filters2", 32, 384)    # Druga warstwa Conv2d
    filters3_2D = trial.suggest_int("Conv2d filters3", 32, 384)    # Trzecia warstwa Conv2d

    l1_1_2D=trial.suggest_float('l1_1_2D', 1e-6, 1e-1, log=True)
    l2_1_2D=trial.suggest_float('l2_1_2D', 1e-6, 1e-1, log=True)
    l1_2_2D=trial.suggest_float('l1_2_2D', 1e-6, 1e-1, log=True)
    l2_2_2D=trial.suggest_float('l2_2_2D', 1e-6, 1e-1, log=True)
    l1_3_2D=trial.suggest_float('l1_3_2D', 1e-6, 1e-1, log=True)
    l2_3_2D=trial.suggest_float('l2_3_2D', 1e-6, 1e-1, log=True)

    # Gałąź SeparableConv1D
    Sep_Conv1D_1 = create_sepconv1d_layer(filters1_1D, activation, l1_1_1D, l2_1_1D)(flattened_input)
    Sep_Conv1D_2 = create_sepconv1d_layer(filters2_1D, activation, l1_2_1D, l2_2_1D)(Sep_Conv1D_1)
    Sep_Conv1D_3 = create_sepconv1d_layer(filters3_1D, activation, l1_3_1D, l2_3_1D)(Sep_Conv1D_2)
    # Pooling 
    Sep_pool_Conv1D = GlobalAveragePooling1D()(Sep_Conv1D_3)    

    # Gałąź Conv2D
        ## Pierwsza warstwa
    conv1 = create_conv2d_layer(filters1_2D, activation, l1_1_2D, l2_1_2D)(image_input)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        ## Druga warstwazz
    conv2 = create_conv2d_layer(filters2_2D, activation, l1_2_2D, l2_2_2D)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        ## Trzecia warstwa
    conv3 = create_conv2d_layer(filters3_2D, activation, l1_3_2D, l2_3_2D)(pool2)
    pool3 = MaxPool2D(pool_size=(3, 3))(conv3)  
    
    flat_cnn = Flatten()(pool3)

    #Dense gałąź SeparableConv1D
    dense1 = Dense(units=trial.suggest_int('dense SeparableConv1D', 32, 384), activation=activation)(Sep_pool_Conv1D)
    dense1 = Dropout(rate=trial.suggest_float('dropout_rate_1', 0.0, 0.75))(dense1)  

    #Dense gałąź Conv2D    
    dense2 = Dense(units=trial.suggest_int('dense Conv2D', 32, 384), activation=activation)(flat_cnn)
    dense2 = Dropout(rate=trial.suggest_float('dropout_rate_2', 0.0, 0.75))(dense2)
    
    # Połączenie gałęzi
    merged = Concatenate()([dense1, dense2])

    merged = Reshape((merged.shape[1], 1))(merged)

    # Attention Layer
    attention_output = Attention()([merged, merged])
    attention_output = Reshape((attention_output.shape[1],))(attention_output)
    
    #Dense
    dense3 = Dense(units=trial.suggest_int('dense przed Output', 36, 384), activation=activation)(attention_output)
    dense3 = Dropout(rate=trial.suggest_float('dropout przed Output', 0.0, 0.75))(dense3)

    #Output
    output = Dense(10, activation='softmax')(dense3)
    model = Model(inputs=[image_input, flattened_input], outputs=output)
    
    #tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    #model.summary()
    
    # ReduceLROnPlateau
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.6, step=0.1),
        patience=trial.suggest_int("reduce_lr_patience", 2, 14),
        min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-4, log=True)
    )

    # Optymalizator
    optimizer_name = trial.suggest_categorical('optimizer', ['AdamW', 'Lion'])
    optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
    # Początkową wartość learning rate
    lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
    optimizer = optimizer_class(learning_rate=lr_initial)  

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=6, start_from_epoch=12, min_delta=0.0001)
    #log_dir = "logs/"  # Katalog, w którym będą zapisywane dane TensorBoard
    #tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1 )

    # Trenowanie modelu
    history = model.fit(
        [X_train_image, X_train_reduced], 
        y_train_filters,
        epochs=trial.suggest_int('epochs', 20, 90),
        batch_size=trial.suggest_int('batch_size', 164, 768),
        validation_data=([X_val_image, X_val_reduced], y_val_filters),
        verbose=1,
        callbacks=[early_stopping, reduce_lr],# , tensorboard
    )

    best_val_accuracy = history.history['val_categorical_accuracy'][-1]

    # Ewaluacja modelu na danych testowych
    #loss, accuracy = model.evaluate([X_test, X_test], y_test_ENC, verbose=1)
    #print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

    del model
    return best_val_accuracy

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=200, n_jobs=1, gc_after_trial=True) ## Czy keras działa na wszystkich wątkach, ale optuna na pojedynczym. 


# Najlepsze parametry i wynik
print("Najlepsze parametry:", study.best_params)
print("Najlepsza dokładność:", study.best_value)

#[I 2024-10-25 02:17:14,226] Trial 55 finished with value:  0.9590833187103271 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 23,  'Sep-Conv2d filters2': 58,  'Sep-Conv2d filters3': 85, 'l1_1_1D': 0.0002263330199021373,   'l2_1_1D': 4.2547366301065874e-06,  'l1_2_1D': 1.8831463765106328e-05,  'l2_2_1D': 2.6849906489521574e-06, 'l1_3_1D': 5.376750360962702e-06, 'l2_3_1D': 4.5507289814394546e-05, 'Conv2d filters1': 27,  'Conv2d filters2': 75, 'Conv2d filters3': 66,  'l1_1_2D': 0.03570924628671723,    'l2_1_2D': 0.00013702268815391088,'l1_2_2D': 0.004180188847206343,   'l2_2_2D': 0.015697748113667306,   'l1_3_2D': 1.1093102758419453e-06, 'l2_3_2D': 2.3039556898400452e-05, 'dense SeparableConv1D': 113, 'dropout_rate_1': 0.25988069757258775, 'dense Conv2D': 55,  'dropout_rate_2': 0.1729409318039701,   'dense przed Output': 213, 'dropout przed Output': 0.692278722477647,    'reduce_lr_factor': 0.1, 'reduce_lr_patience': 8, 'reduce_lr_min_lr': 1.2382721420683838e-05, 'optimizer': 'AdamW', 'lr_initial': 0.007665780845145814, 'epochs': 81, 'batch_size': 173}. Best is trial 55 with value: 0.9590833187103271.
#[I 2024-10-25 05:23:25,094] Trial 82 finished with value:  0.9622499942779541 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 22,  'Sep-Conv2d filters2': 28,  'Sep-Conv2d filters3': 100, 'l1_1_1D': 6.186293277794836e-05,  'l2_1_1D': 0.0003913398305461908,   'l1_2_1D': 1.5927923474696962e-06,  'l2_2_1D': 7.009282960520817e-06,  'l1_3_1D': 7.676137877709806e-06, 'l2_3_1D': 0.00022663346915615475, 'Conv2d filters1': 24,  'Conv2d filters2': 74, 'Conv2d filters3': 92,  'l1_1_2D': 0.0993169614612255,     'l2_1_2D': 0.0006883072766765725, 'l1_2_2D': 0.0005616491418975441,  'l2_2_2D': 1.0689411586574192e-05, 'l1_3_2D': 4.1278686535916746e-05, 'l2_3_2D': 4.837146281888261e-06,  'dense SeparableConv1D': 155, 'dropout_rate_1': 0.0509776183623675,  'dense Conv2D': 29,  'dropout_rate_2': 0.024717745882008837, 'dense przed Output': 178, 'dropout przed Output': 0.7445169215013963, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 4, 'reduce_lr_min_lr': 6.725637729646196e-06,  'optimizer': 'AdamW', 'lr_initial': 0.01728017154953551,  'epochs': 85, 'batch_size': 191}. Best is trial 82 with value: 0.9622499942779541.
#[I 2024-10-25 15:22:24,767] Trial 142 finished with value: 0.9635000228881836 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 22,  'Sep-Conv2d filters2': 28,  'Sep-Conv2d filters3': 107, 'l1_1_1D': 6.120269265952697e-05,  'l2_1_1D': 7.956576920829512e-06,   'l1_2_1D': 1.2494724596667829e-06,  'l2_2_1D': 7.6581362167544e-06,    'l1_3_1D': 0.0039961697402018225, 'l2_3_1D': 0.00021250335195961356, 'Conv2d filters1': 28,  'Conv2d filters2': 72, 'Conv2d filters3': 99 , 'l1_1_2D': 0.07544375319709569,    'l2_1_2D': 0.000936229620640654,  'l1_2_2D': 0.001250431002984649,   'l2_2_2D': 0.01324640444949233,    'l1_3_2D': 0.0012955495310992596,  'l2_3_2D': 6.618279886291623e-06,  'dense SeparableConv1D': 151, 'dropout_rate_1': 0.08650725184529029, 'dense Conv2D': 29,  'dropout_rate_2': 0.0500476697674726,   'dense przed Output': 205, 'dropout przed Output': 0.7360088478595872,   'reduce_lr_factor': 0.4, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 9.207463977521836e-07,  'optimizer': 'AdamW', 'lr_initial': 0.01209682198565378,  'epochs': 88, 'batch_size': 228}. Best is trial 142 with value: 0.9635000228881836.
#[I 2024-11-01 00:07:23,390] Trial 24 finished with value:  0.9449999928474426 and parameters: {'activation': 'mish',        'Sep-Conv2d filters1': 123, 'Sep-Conv2d filters2': 245, 'Sep-Conv2d filters3': 289, 'l1_1_1D': 0.08989218874411514,    'l2_1_1D': 1.1472596089503972e-06,  'l1_2_1D': 0.0026057385621594867,   'l2_2_1D': 0.00018978883951458408, 'l1_3_1D': 1.191033349842715e-06, 'l2_3_1D': 0.00014292949482563498, 'Conv2d filters1': 308, 'Conv2d filters2': 81, 'Conv2d filters3': 326, 'l1_1_2D': 2.9840369417992957e-06, 'l2_1_2D': 0.0014037071972283193, 'l1_2_2D': 0.0002640492511862146,  'l2_2_2D': 0.08499221386872775,    'l1_3_2D': 0.020374031922578733,   'l2_3_2D': 0.00010028036351417339, 'dense SeparableConv1D': 379, 'dropout_rate_1': 0.0544686509758308,  'dense Conv2D': 138, 'dropout_rate_2': 0.09593865740145555,  'dense przed Output': 384, 'dropout przed Output': 0.6382663359384837, 'reduce_lr_factor': 0.3, 'reduce_lr_patience': 4, 'reduce_lr_min_lr': 7.741233462096205e-06,  'optimizer': 'AdamW', 'lr_initial': 0.006707953904631179, 'epochs': 65, 'batch_size': 257}. Best is trial 24 with value: 0.9449999928474426.
#[I 2024-11-05 15:57:39,996] Trial 101 finished with value: 0.9495833516120911 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 148, 'Sep-Conv2d filters2': 32,  'Sep-Conv2d filters3': 63, 'l1_1_1D': 0.0004897725865476358,   'l2_1_1D': 0.0003049226492905848,   'l1_2_1D': 5.00894966144464e-06,    'l2_2_1D': 0.00014051697363473006, 'l1_3_1D': 0.051135669099956614,  'l2_3_1D': 0.0009950758155242705,  'Conv2d filters1': 52,  'Conv2d filters2': 58, 'Conv2d filters3': 249, 'l1_1_2D': 0.014787666006730659,   'l2_1_2D': 0.0005948328558539133, 'l1_2_2D': 5.343054689024963e-06,  'l2_2_2D': 4.5277461875969285e-06, 'l1_3_2D': 1.9548002422161054e-06, 'l2_3_2D': 0.0009410443070360079,  'dense SeparableConv1D': 319, 'dropout_rate_1': 0.2533943711642646,  'dense Conv2D': 311, 'dropout_rate_2': 0.10208101037728201,  'dense przed Output': 90,  'dropout przed Output': 0.481006546242034,   'reduce_lr_factor': 0.5, 'reduce_lr_patience': 2, 'reduce_lr_min_lr': 5.559247664816983e-05,  'optimizer': 'AdamW', 'lr_initial': 0.00964928371074687,  'epochs': 66, 'batch_size': 423}. Best is trial 101 with value: 0.9495833516120911.
#[I 2024-11-06 02:29:49,053] Trial 121 finished with value: 0.9535833597183228 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 119, 'Sep-Conv2d filters2': 49,  'Sep-Conv2d filters3': 158, 'l1_1_1D': 0.00023280964689116444, 'l2_1_1D': 0.0008751788463496611,   'l1_2_1D': 0.04610519254480619,     'l2_2_1D': 2.4445376899197913e-06, 'l1_3_1D': 0.06955230495640688,   'l2_3_1D': 0.004678025581652909,   'Conv2d filters1': 37,  'Conv2d filters2': 54, 'Conv2d filters3': 253, 'l1_1_2D': 0.016409226434426858,  'l2_1_2D': 0.00015292748828121323, 'l1_2_2D': 1.177904875672071e-05,  'l2_2_2D': 2.6668468645182726e-06, 'l1_3_2D': 6.7648129463989756e-06, 'l2_3_2D': 0.0007509210891800366,  'dense SeparableConv1D': 347, 'dropout_rate_1': 0.2855797128973145,  'dense Conv2D': 368, 'dropout_rate_2': 0.07539865516979521,  'dense przed Output': 73,  'dropout przed Output': 0.5059616088424707, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 2, 'reduce_lr_min_lr': 3.887676551167433e-05, 'optimizer': 'AdamW', 'lr_initial': 0.00852437932688485, 'epochs': 73, 'batch_size': 407}. Best is trial 121 with value: 0.9535833597183228.